# URL de las distintas fuentes de datos

## PRODUCCIÓN ANUAL AGRÍCOLA

https://nube.siap.gob.mx/cierreagricola/

## PRODUCCIÓN MENSUAL AGRÍCOLA

https://nube.siap.gob.mx/avance_agricola/

http://infosiap.siap.gob.mx:8080/agricola_siap_gobmx/ResumenProducto.do

## INTENCION DE SIEMBRA

https://nube.siap.gob.mx/agroprograma/

## SISTEMA DE INFORMACIÓN AGROALIMENTARIA DE CONSULTA

http://infosiap.siap.gob.mx/gobmx/datosAbiertos_a.php

## ESTADÍSTICA DE USO TECNOLÓGICO Y DE SERVICIOS EN LA SUPERFICIE AGRÍCOLA

Fertilizada: https://nube.siap.gob.mx/index.php/s/kTLGLipCBEBiEFM/download

Semilla mejorada y semilla criolla: https://nube.siap.gob.mx/index.php/s/OjKcZZL8QClaK6U/download

Mecanizada: https://nube.siap.gob.mx/index.php/s/P7yK8NWmppStv7i/download

Riego: https://nube.siap.gob.mx/index.php/s/cz6nETNW4b0krzD/download

Sanidad: https://nube.siap.gob.mx/index.php/s/0l0Vw6VUf8fQ0dI/download

Asistencia técnica: https://nube.siap.gob.mx/index.php/s/EHaUuYSEnHqHLqA/download

## INFORMACIÓN AGRÍCOLA DE GÉNERO

2015 - 2017: https://www.gob.mx/cms/uploads/attachment/file/399835/Estadistica_de_G_nero_2017.pdf

2016 - 2018: https://www.gob.mx/cms/uploads/attachment/file/515428/Participaci_n_de_mujeres_en_valor_2016_17_18.pdf

2017 - 2019: https://www.gob.mx/cms/uploads/attachment/file/630415/Estadistica_de_Genero_2019.pdf

## MONOGRAFÍAS DE CULTIVOS

https://www.gob.mx/siap/documentos/monografias



# Datos anuales

In [ ]:
# Importamos librerías a utilizar
import pandas as pd
import requests
from bs4 import BeautifulSoup
from io import StringIO

In [ ]:
"""Código para almacenar el contenido de los CSVs en DataFrames.
   A su vez, estos DataFrames serán almacenados en un diccionario.
   Cada elemento del diccionario será un DataFrame asociado a una key con el año
   respectivo de los datos"""

# URL base de la página web que contiene los enlaces
base_url = "http://infosiap.siap.gob.mx/gobmx/datosAbiertos_a.php"

# Realizar una solicitud GET para obtener el contenido de la página web
response = requests.get(base_url)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    # Parsear el contenido HTML de la página web (web scrapping)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Encontrar todos los HTML elements con enlaces que apuntan a archivos CSV
    links = soup.find_all('a', href=lambda href: href and href.endswith('.csv'))

    # Diccionario para almacenar los dataframes
    dataframes_dict = {}

    # Iterar sobre los enlaces y descargar los archivos CSV
    for link in links:
        href = link['href']                       # Extraer los enlaces de los HTML elements
        year = href.split('_')[-1].split('.')[0]  # Obtener el año del nombre del archivo

        # URL completa para el archivo CSV
        csv_url = f"http://infosiap.siap.gob.mx/gobmx/{href}"

        # Solicitud GET para descargar el archivo CSV
        csv_response = requests.get(csv_url)

        # Verificar si la solicitud fue exitosa
        if csv_response.status_code == 200:
            # Leer el contenido CSV en un dataframe
            csv_content = csv_response.text
            df = pd.read_csv(StringIO(csv_content))

            # Almacenar el dataframe en el diccionario con el año como key
            dataframes_dict[year] = df

            # Notificación de éxito
            print(f"Año {year} descargado con éxito")

        # Notificación de fallo
        else:
            print(f"No se pudo obtener el archivo CSV para el año {year}")

# Notificación de fallo
else:
    print("No se pudo obtener el contenido de la página web")

<ipython-input-22-3148553f859c>:38: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(StringIO(csv_content))


Año 2022 descargado con éxito
Año 2021 descargado con éxito
Año 2020 descargado con éxito
Año 2019 descargado con éxito
Año 2018 descargado con éxito
Año 2017 descargado con éxito
Año 2016 descargado con éxito
Año 2015 descargado con éxito
Año 2014 descargado con éxito
Año 2013 descargado con éxito
Año 2012 descargado con éxito
Año 2011 descargado con éxito
Año 2010 descargado con éxito


<ipython-input-22-3148553f859c>:38: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(StringIO(csv_content))


Año 2009 descargado con éxito
Año 2008 descargado con éxito
Año 2007 descargado con éxito
Año 2006 descargado con éxito
Año 2005 descargado con éxito
Año 2004 descargado con éxito
Año 2003 descargado con éxito
Año 2002 descargado con éxito
Año 2001 descargado con éxito
Año 2000 descargado con éxito
Año 1999 descargado con éxito
Año 1998 descargado con éxito
Año 1997 descargado con éxito
Año 1996 descargado con éxito
Año 1995 descargado con éxito
Año 1994 descargado con éxito
Año 1993 descargado con éxito
Año 1992 descargado con éxito
Año 1991 descargado con éxito
Año 1990 descargado con éxito
Año 1989 descargado con éxito
Año 1988 descargado con éxito
Año 1987 descargado con éxito
Año 1986 descargado con éxito
Año 1985 descargado con éxito
Año 1984 descargado con éxito
Año 1983 descargado con éxito
Año 1982 descargado con éxito
Año 1981 descargado con éxito
Año 1980 descargado con éxito
Año seguimiento descargado con éxito
Año seguimiento descargado con éxito


In [ ]:
# Podemos acceder a los dataframes por año usando el diccionario
dataframes_dict['2022'].head()

,Anio,Idestado,Nomestado,Idddr,Nomddr,Idcader,Nomcader,Idmunicipio,Nommunicipio,Idciclo,...,Nomunidad,Idcultivo,Nomcultivo,Sembrada,Cosechada,Siniestrada,Volumenproduccion,Rendimiento,Preciomediorural,Valorproduccion
0,2022,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,1,...,Tonelada,5490000,Avena forrajera en verde,1180.0,1180.0,0.0,31270,26.50,565.13,17671615.1
1,2022,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,1,...,Tonelada,5670000,Brócoli,45.0,45.0,0.0,810,18.00,3805.33,3082317.3
2,2022,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,1,...,Tonelada,5900000,Cebada forrajera en verde,10.0,10.0,0.0,280,28.00,590.00,165200.0
3,2022,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,1,...,Tonelada,9090000,Triticale forrajero en verde,300.0,300.0,0.0,9180,30.60,550.00,5049000.0
4,2022,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,1,...,Tonelada,15050000,Pastos y praderas,527.5,527.5,0.0,26290.6,49.84,520.00,13671112.0


In [ ]:
# Podemos inspeccionar sus variables
dataframes_dict['2022'].dtypes

Anio                    int64
Idestado                int64
Nomestado              object
Idddr                   int64
Nomddr                 object
Idcader                 int64
Nomcader               object
Idmunicipio             int64
Nommunicipio           object
Idciclo                 int64
Nomcicloproductivo     object
Idmodalidad             int64
Nommodalidad           object
Idunidadmedida          int64
Nomunidad              object
Idcultivo               int64
Nomcultivo             object
Sembrada              float64
Cosechada             float64
Siniestrada           float64
Volumenproduccion      object
Rendimiento           float64
Preciomediorural      float64
Valorproduccion       float64
dtype: object

In [ ]:
"""Procedemos a renombras aquellas columnas que nos interesan."""

# Columnas que queremos mantener
cols = ('Anio', 'Nomestado', 'Nommunicipio', 'Nomcicloproductivo', 'Nommodalidad',
       'Nomunidad', 'Nomcultivo', 'Sembrada', 'Cosechada',
       'Siniestrada', 'Volumenproduccion', 'Rendimiento', 'Preciomediorural',
       'Valorproduccion')

# Nuevos nombres de las columnas mantenidas
col_names = ('Año', 'Estado', 'Municipio', 'Ciclo_productivo', 'Modalidad',
             'Unidad', 'Cultivo', 'Superficie_sembrada (ha)', 'Superficie_cosechada (ha)',
             'Superficie_siniestrada (ha)', 'Volumen_produccion', 'Rendimiento',
             'Precio_medio_rural', 'Valor_produccion')

# Diccionario para almacenar los dataframes con columnas renombradas
renamed_dataframes_dict = {}

# Itera a través de los dataframes originales
for year, df in dataframes_dict.items():
    # Copia el dataframe para no modificar el original
    renamed_df = df.copy()

    # Renombra las columnas
    renamed_df = renamed_df.rename(columns=dict(zip(cols, col_names)))

    # Almacena el dataframe renombrado en el nuevo diccionario
    renamed_dataframes_dict[year] = renamed_df

    # Notificación de éxito
    print(f"Año {year} renombrado con éxito")



Año 2022 renombrado con éxito
Año 2021 renombrado con éxito
Año 2020 renombrado con éxito
Año 2019 renombrado con éxito
Año 2018 renombrado con éxito
Año 2017 renombrado con éxito
Año 2016 renombrado con éxito
Año 2015 renombrado con éxito
Año 2014 renombrado con éxito
Año 2013 renombrado con éxito
Año 2012 renombrado con éxito
Año 2011 renombrado con éxito
Año 2010 renombrado con éxito
Año 2009 renombrado con éxito
Año 2008 renombrado con éxito
Año 2007 renombrado con éxito
Año 2006 renombrado con éxito
Año 2005 renombrado con éxito
Año 2004 renombrado con éxito
Año 2003 renombrado con éxito
Año 2002 renombrado con éxito
Año 2001 renombrado con éxito
Año 2000 renombrado con éxito
Año 1999 renombrado con éxito
Año 1998 renombrado con éxito
Año 1997 renombrado con éxito
Año 1996 renombrado con éxito
Año 1995 renombrado con éxito
Año 1994 renombrado con éxito
Año 1993 renombrado con éxito
Año 1992 renombrado con éxito
Año 1991 renombrado con éxito
Año 1990 renombrado con éxito
Año 1989 r

In [ ]:
# Inspeccionamos los dataframes y notamos que aún están presentes columnas que no nos interesan
renamed_dataframes_dict['2020'].dtypes

Año                              int64
Idestado                         int64
Estado                          object
Idddr                            int64
Nomddr                          object
Idcader                          int64
Nomcader                        object
Idmunicipio                      int64
Municipio                       object
Idciclo                          int64
Ciclo_productivo                object
Idmodalidad                      int64
Modalidad                       object
Idunidadmedida                   int64
Unidad                          object
Idcultivo                        int64
Nomcultivo Sin Um               object
Superficie_sembrada (ha)       float64
Superficie_cosechada (ha)      float64
Superficie_siniestrada (ha)    float64
Volumen_produccion             float64
Rendimiento                    float64
Precio                         float64
Valor_produccion               float64
dtype: object

In [ ]:
"""Algunos DataFrames originales poseen la columna 'Nomcultivo Sin Um' en lugar de 'Nomcultivo'.
   Por lo tanto, tomamos en consideración estos casos para realizar correctamente el cambio de
   nombre a 'Cultivo'.
   Por otro lado, notamos que las columnas que no nos interesan poseen algunas de las siguientes
   características:

   - Terminan en 'cader'
   - Empiezan con 'Id'
   - Terminan en 'ddr'

   Eliminamos aquellas columnas que cumplan con alguna de estas condiciones."""

# Definir una función para renombrar las columnas con "cultivo" en el nombre
def renombrar_cultivo(columna):
    return "Cultivo" if "cultivo" in columna.lower() else columna

# Diccionario para almacenar los dataframes con las transformaciones aplicadas
transformed_dataframes_dict = {}

# Itera a través de los dataframes renombrados
for year, renamed_df in renamed_dataframes_dict.items():
    # Copia el dataframe para no modificar el original
    transformed_df = renamed_df.copy()

    # Elimina las columnas que comienzan con "Id", terminan con "cader" o contienen "ddr"
    columns_to_drop = [col for col in transformed_df.columns if col.startswith("Id") or col.endswith("cader") or col.endswith("ddr")]
    transformed_df = transformed_df.drop(columns=columns_to_drop)

    # Renombra las columnas que contienen "cultivo"
    transformed_df = transformed_df.rename(columns=renombrar_cultivo)

    # Almacena el dataframe transformado en el nuevo diccionario
    transformed_dataframes_dict[year] = transformed_df

    # Notificación de éxito
    print(f"Año {year} transformado con éxito")

Año 2022 transformado con éxito
Año 2021 transformado con éxito
Año 2020 transformado con éxito
Año 2019 transformado con éxito
Año 2018 transformado con éxito
Año 2017 transformado con éxito
Año 2016 transformado con éxito
Año 2015 transformado con éxito
Año 2014 transformado con éxito
Año 2013 transformado con éxito
Año 2012 transformado con éxito
Año 2011 transformado con éxito
Año 2010 transformado con éxito
Año 2009 transformado con éxito
Año 2008 transformado con éxito
Año 2007 transformado con éxito
Año 2006 transformado con éxito
Año 2005 transformado con éxito
Año 2004 transformado con éxito
Año 2003 transformado con éxito
Año 2002 transformado con éxito
Año 2001 transformado con éxito
Año 2000 transformado con éxito
Año 1999 transformado con éxito
Año 1998 transformado con éxito
Año 1997 transformado con éxito
Año 1996 transformado con éxito
Año 1995 transformado con éxito
Año 1994 transformado con éxito
Año 1993 transformado con éxito
Año 1992 transformado con éxito
Año 1991

In [ ]:
# Podemos acceder a cada dataframe por año de la misma manera que antes
transformed_dataframes_dict['2016']

,Año,Estado,Municipio,Ciclo_productivo,Modalidad,Unidad,Cultivo,Superficie_sembrada (ha),Superficie_cosechada (ha),Superficie_siniestrada (ha),Volumen_produccion,Rendimiento,Precio,Valor_produccion
0,2016,Aguascalientes,Aguascalientes,Otoño-Invierno,Riego,Tonelada,Avena forrajera en verde,765.0,765.0,0.0,20865.75,27.28,595.45,12424510.84
1,2016,Aguascalientes,Aguascalientes,Otoño-Invierno,Riego,Tonelada,Cebada forrajera en verde,35.0,35.0,0.0,945.00,27.00,570.00,538650.00
2,2016,Aguascalientes,Aguascalientes,Otoño-Invierno,Riego,Tonelada,Triticale forrajero en verde,611.0,611.0,0.0,20446.00,33.46,593.17,12127953.82
3,2016,Aguascalientes,Aguascalientes,Otoño-Invierno,Riego,Tonelada,Pastos y praderas,692.0,692.0,0.0,34912.70,50.45,601.15,20987769.60
4,2016,Aguascalientes,Aguascalientes,Primavera-Verano,Riego,Tonelada,Avena forrajera en verde,28.0,28.0,0.0,746.40,26.66,545.91,407467.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36924,2016,Zacatecas,Jalpa,Perennes,Riego,Tonelada,Tuna,40.0,40.0,0.0,508.00,12.70,4800.00,2438400.00
36925,2016,Zacatecas,Jalpa,Perennes,Riego,Tonelada,Uva,17.0,0.0,0.0,0.00,0.00,0.00,0.00
36926,2016,Zacatecas,Jalpa,Perennes,Riego,Tonelada,Pastos y praderas,147.0,147.0,0.0,9391.50,63.89,591.90,5558824.00
36927,2016,Zacatecas,Jalpa,Perennes,Temporal,Tonelada,Agave,110.0,85.0,0.0,8270.50,97.30,3800.00,31427900.00


# Datos mensuales

#### Utilizamos el código de Gaytán para obtener el XML en el cual está contenida la tabla que se genera de manera dinámica en la pagina. Los elementos en "'xajaxargs[]" determinan los filtros que aplicaremos. Podemos utilizar el JSON creado por Victor para ajustar los filtros.


In [ ]:
# Importamos librerías a utilizar
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
# Instalamos paquete
#pip install lxml

In [84]:
# Comando para ignorar warnings (salen en cada request)
import warnings
warnings.filterwarnings('ignore')

In [85]:
# Endpoint
url = "https://nube.siap.gob.mx/avance_agricola/"

payload = {'xajax': 'reporte', # Para obtener la tabla
'xajaxr': '1696449941927', # Timestamp UNIX
'xajaxargs[]': [
    '1', # 1: Desglose por estados, 2: Cultivo total
    '2023', # Anio
    '5', # ID Ciclo
    '3', # ID Modalidad
    '0', # ID Estado (0: Nacional)
    '--',
    '--',
    '7', # ID Cultivo
    '200201',
    '0',
    '1', # 1: Por municipio
    'undefined',
    'undefined',
    'undefined',
    '8' # Valor del mes: va de 1 a 8, con 1 siendo Enero y 8 Agosto
    ]
}

headers = {
  'Cookie': 'PHPSESSID=45ri2k73cbp2iptcrufu88p360'
}

# Hacemos el request
response = requests.request("POST", url, headers=headers, data=payload, verify=False)
response.encoding='ISO-8859-1'
# Imprimimos el status de nuestro request
print(f"Status del HTTP request: {response.status_code}")

Status del HTTP request: 200


#### Convertimos el código anterior en una función para poder utilizarla de manera repetida (por lo pronto, solo dejamos como argumentos el año y el mes).

In [86]:
def request_xml(año, mes):
  """Función para solicitar el XML a la página del SIAP, especificando el año y
     més agrícola de nuestro interés."""

  # Endpoint
  url = "https://nube.siap.gob.mx/avance_agricola/"

  payload = {'xajax': 'reporte', # Para obtener la tabla
  'xajaxr': '1696449941927', # Timestamp UNIX
  'xajaxargs[]': [
      '1', # 1: Desglose por estados, 2: Cultivo total
      str(año), # Anio
      '5', # ID Ciclo
      '3', # ID Modalidad
      '0', # ID Estado (0: Nacional)
      '--',
      '--',
      '7', # ID Cultivo
      '200201',
      '0',
      '1', # 1: Por municipio
      'undefined',
      'undefined',
      'undefined',
      str(mes) # Valor del mes: va de 1 a 8, con 1 siendo Enero y 8 Agosto
      ]
  }

  headers = {
    'Cookie': 'PHPSESSID=45ri2k73cbp2iptcrufu88p360'
  }

  # Hacemos el request
  response = requests.request("POST", url, headers=headers, data=payload, verify=False)
  response.encoding='ISO-8859-1'

  # Regresamos el XML como string
  return response.text

#### Creamos un Dataframe a partir del XML.

In [87]:
# Usamos nuestra nueva función para obtener el XML de Enero del 2020
xml_data = request_xml(2020, 8)

# Parseamos el XML con BeautifulSoup usando el analizador XML (lxml)
soup = BeautifulSoup(xml_data, 'lxml');

# Encontramos la tabla por su ID
tabla = soup.find('table', id='Resultados-reporte')

if tabla:
    # Inicializamos listas para almacenar los datos
    datos = []

    # Iteramos a través de las filas de la tabla
    for fila in tabla.find_all('tr')[1:]:  # Ignoramos la primera fila de encabezados
        columnas = fila.find_all('td')
        if len(columnas) == 0:  # Ignoramos las filas vacías
            continue

        # Extraemos los datos relevantes de cada columna
        #posicion = columnas[0].get_text()
        entidad = columnas[1].get_text()
        municipio = columnas[2].get_text()
        superficie_sembrada = columnas[3].get_text()
        superficie_cosechada = columnas[4].get_text()
        superficie_siniestrada = columnas[5].get_text()
        produccion = columnas[6].get_text()
        rendimiento = columnas[7].get_text()

        # Agregamos los datos a la lista
        datos.append([
            entidad, municipio,
            superficie_sembrada, superficie_cosechada,
            superficie_siniestrada, produccion, rendimiento
        ])

    # Creamos un DataFrame de Pandas con los datos
    columnas = ["Entidad", "Municipio", "Superficie Sembrada", "Superficie Cosechada", "Superficie Siniestrada", "Produccion", "Rendimiento"]
    df = pd.DataFrame(datos, columns=columnas)

    # Imprimimos mensaje de éxito
    print("El dataframe se creó con éxito")
else:
    print("La tabla no se encontró.")

# Inspeccionamos el dataframe
df


El dataframe se creó con éxito


,Entidad,Municipio,Superficie Sembrada,Superficie Cosechada,Superficie Siniestrada,Produccion,Rendimiento
0,Aguascalientes,Calvillo,55.85,0.00,0.00,0.00,0
1,Baja California,Ensenada,64.50,0.00,0.00,0.00,0
2,Baja California Sur,Comondú,16.00,0.00,0.00,0.00,0
3,Baja California Sur,Mulegé,6.00,0.00,0.00,0.00,0
4,Baja California Sur,La Paz,9.00,9.00,0.00,45.90,5.10
...,...,...,...,...,...,...,...
630,Zacatecas,Jerez,14.00,0.00,0.00,0.00,0
631,Zacatecas,Juchipila,6.50,0.00,0.00,0.00,0
632,Zacatecas,Mezquital del Oro,6.00,0.00,0.00,0.00,0
633,Zacatecas,Moyahua de Estrada,9.00,0.00,0.00,0.00,0


#### Convertimos el código anterior en una función para poder utilizarla de manera repetida (por lo pronto, solo dejamos como argumentos el año y el mes).

In [88]:
def xml_to_df(año, mes):
  """Función para convertir el XML en un Dataframe."""

  # Usamos nuestra nueva función para obtener el XML de Enero del 2020
  xml_data = request_xml(año, mes)

  # Parseamos el XML con BeautifulSoup usando el analizador XML (lxml)
  soup = BeautifulSoup(xml_data, 'lxml');

  # Encontramos la tabla por su ID
  tabla = soup.find('table', id='Resultados-reporte')

  if tabla:
      # Inicializamos listas para almacenar los datos
      datos = []

      # Iteramos a través de las filas de la tabla
      for fila in tabla.find_all('tr')[1:]:  # Ignoramos la primera fila de encabezados
          columnas = fila.find_all('td')
          if len(columnas) == 0:  # Ignoramos las filas vacías
              continue

          # Extraemos los datos relevantes de cada columna
          #posicion = columnas[0].get_text()
          entidad = columnas[1].get_text()
          municipio = columnas[2].get_text()
          superficie_sembrada = columnas[3].get_text()
          superficie_cosechada = columnas[4].get_text()
          superficie_siniestrada = columnas[5].get_text()
          produccion = columnas[6].get_text()
          rendimiento = columnas[7].get_text()

          # Agregamos los datos a la lista
          datos.append([
              entidad, municipio,
              superficie_sembrada, superficie_cosechada,
              superficie_siniestrada, produccion, rendimiento
          ])

      # Creamos un DataFrame de Pandas con los datos
      columnas = ["Entidad", "Municipio", "Superficie Sembrada", "Superficie Cosechada", "Superficie Siniestrada", "Produccion", "Rendimiento"]
      df = pd.DataFrame(datos, columns=columnas)

      # Regresamos el dataframe
      return df

#### Empleando estas dos funciones creamos un diccionario con el siguiente formato:

- key: "año_mes"
- value: el dataframe correspodiente al key

In [91]:
# Diccionario vacío
month_df_dict = {}

# Iteración sobre los años y meses de interés
for año in range(2020,2023):
  for mes in range(1,9):

    # Añadir elemento al diccionario
    month_df_dict[str(año) + "_" + str(mes)] = xml_to_df(año, mes)

    # Imprimimos mensaje de éxito
    print("El dataframe " + str(año) + "_" + str(mes) + " se creó con éxito")

El dataframe 2020_1 se creó con éxito
El dataframe 2020_2 se creó con éxito
El dataframe 2020_3 se creó con éxito
El dataframe 2020_4 se creó con éxito
El dataframe 2020_5 se creó con éxito
El dataframe 2020_6 se creó con éxito
El dataframe 2020_7 se creó con éxito
El dataframe 2020_8 se creó con éxito
El dataframe 2021_1 se creó con éxito
El dataframe 2021_2 se creó con éxito
El dataframe 2021_3 se creó con éxito
El dataframe 2021_4 se creó con éxito
El dataframe 2021_5 se creó con éxito
El dataframe 2021_6 se creó con éxito
El dataframe 2021_7 se creó con éxito
El dataframe 2021_8 se creó con éxito
El dataframe 2022_1 se creó con éxito
El dataframe 2022_2 se creó con éxito
El dataframe 2022_3 se creó con éxito
El dataframe 2022_4 se creó con éxito
El dataframe 2022_5 se creó con éxito
El dataframe 2022_6 se creó con éxito
El dataframe 2022_7 se creó con éxito
El dataframe 2022_8 se creó con éxito


In [92]:
# Analizamos las keys del diccionario
month_df_dict.keys()

dict_keys(['2020_1', '2020_2', '2020_3', '2020_4', '2020_5', '2020_6', '2020_7', '2020_8', '2021_1', '2021_2', '2021_3', '2021_4', '2021_5', '2021_6', '2021_7', '2021_8', '2022_1', '2022_2', '2022_3', '2022_4', '2022_5', '2022_6', '2022_7', '2022_8'])

In [107]:
# Imprimimos un dataframe cualquiera
month_df_dict['2022_7']

,Entidad,Municipio,Superficie Sembrada,Superficie Cosechada,Superficie Siniestrada,Produccion,Rendimiento
0,Aguascalientes,Calvillo,55.85,0.00,0.00,0.00,0
1,Baja California,Ensenada,44.00,0.00,0.00,0.00,0
2,Baja California Sur,Comondú,16.00,0.00,0.00,0.00,0
3,Baja California Sur,Mulegé,6.00,0.00,0.00,0.00,0
4,Baja California Sur,La Paz,9.00,9.00,0.00,46.08,5.12
...,...,...,...,...,...,...,...
636,Zacatecas,Jerez,14.00,0.00,0.00,0.00,0
637,Zacatecas,Juchipila,6.50,0.00,0.00,0.00,0
638,Zacatecas,Mezquital del Oro,6.00,0.00,0.00,0.00,0
639,Zacatecas,Moyahua de Estrada,9.00,0.00,0.00,0.00,0


#### Un siguiente paso sería aplicar los filtros respectivos para solo extraer la información de frutas y hortalizas. La siguiente información es útil:

"
De acuerdo con el Atlas agroalimentario del SIAP, los tipos de hortalizas en México se clasifican de la siguiente manera: de raíz (betabel, nabo, rábano, zanahoria); de flor-coles (alcachofa, brócoli, coliflor); de hoja (acelga, apio, espinaca, lechuga, perejil y repollo); de tallos bulbos (ajo, cebolla y papa), y de frutos (ajíes, arveja, berenjena, haba, pepino, pimentón y tomate).

Entre las principales hortalizas que se producen en México se encuentran: el jitomate o tomate rojo, aguacate, cebolla blanca, chile jalapeño, tomate verde y calabaza, sólo por mencionar algunas.
"

"
Tipos de frutas cultivadas en México

Tropicales y subtropicales:

Incluye frutas como plátano, aguacate, mango, piña y papaya.

Su producción en 2020 fue de 10,317,667 toneladas.

Cítricos:

Incluye frutas como naranja, limón, toronja, mandarina y tangerina.

Su producción en 2020 fue de 8,686,667 toneladas.

De pepita y hueso:

Incluye frutas como manzana, durazno, ciruela, aceituna y mamey.

Su producción en 2020 fue de 1,082,366 toneladas.

Bayas

Incluye frutas como fresa, zarzamora, frambuesa y arándano.

Su producción en 2020 fue de 970,080 toneladas.

Vid:

Se refiere a la uva.

Su producción en 2020 fue de 380,001 toneladas.

Nueces:

Incluye frutas como nuez, piñón y pistache.

Su producción en 2020 fue de 174, 830 toneladas

Otras:

Incluye frutas como tuna, tamarindo, pitahaya, pitaya y algarrobo.

Su producción en 2020 fue de 544,956 toneladas.

Sabías que… Las nueces y el coco son semillas del fruto, la caña es un tallo, el melón y la sandía son hortalizas, pero los consumidores los conocen por fruta.

En el ranking internacional, México es:

1° productor de aguacate y zarzamora

2° productor de limón y frambuesa

3° productor de fresa
"

In [109]:
# Creamos listas con los productos que pertenecen a las categorias "hortaliza" y "fruta"
hortalizas = ["betabel", "nabo", "rábano", "zanahoria", "alcachofa", "brócoli", "coliflor", "acelga", "apio", "espinaca", "lechuga", "perejil", "repollo", "ajo", "cebolla", "papa", "ajíes", "arveja", "berenjena", "haba", "pepino", "pimentón", "tomate"]
frutas = ["plátano", "aguacate", "mango", "piña", "papaya", "naranja", "limón", "toronja", "mandarina", "tangerina", "manzana", "durazno", "ciruela", "aceituna", "mamey", "fresa", "zarzamora", "frambuesa", "arándano", "uva", "nuez", "piñón", "pistache", "tuna", "tamarindo", "pitahaya", "pitaya", "algarrobo"]

Nota: estas listas engloban 51 de los 64 cultivos que vienen en la página

#### JSON de Victor

Nota: sí vienen los 64 cultivos que aparecen en la página

In [108]:

victor_dict = {
    "anioagric_2023": "2023",
    "anioagric_2022": "2022",
    "anioagric_2021": "2021",
    "anioagric_2020": "2020",
    "anioagric_2019": "2019",
    "anioagric_2018": "2018",
    "mesagric_Agosto": "8",
    "mesagric_Julio": "7",
    "mesagric_Junio": "6",
    "mesagric_Mayo": "5",
    "mesagric_Abril": "4",
    "mesagric_Marzo": "3",
    "mesagric_Febrero": "2",
    "mesagric_Enero": "1",
    "cicloProd_Otoño - Invierno": "1",
    "cicloProd_Primavera - Verano": "2",
    "cicloProd_Perennes": "3",
    "cicloProd_Año Agrícola (OI + PV)": "4",
    "cicloProd_Ciclicos - Perennes": "5",
    "modalidad_Riego": "1",
    "modalidad_Temporal": "2",
    "modalidad_Riego + Temporal": "3",
    "entidad_Nacional": "0",
    "entidad_Aguascalientes": "1",
    "entidad_Baja California": "2",
    "entidad_Baja California Sur": "3",
    "entidad_Campeche": "4",
    "entidad_Coahuila": "5",
    "entidad_Colima": "6",
    "entidad_Chiapas": "7",
    "entidad_Chihuahua": "8",
    "entidad_Ciudad de México": "9",
    "entidad_Durango": "10",
    "entidad_Guanajuato": "11",
    "entidad_Guerrero": "12",
    "entidad_Hidalgo": "13",
    "entidad_Jalisco": "14",
    "entidad_México": "15",
    "entidad_Michoacán": "16",
    "entidad_Morelos": "17",
    "entidad_Nayarit": "18",
    "entidad_Nuevo León": "19",
    "entidad_Oaxaca": "20",
    "entidad_Puebla": "21",
    "entidad_Querétaro": "22",
    "entidad_Quintana Roo": "23",
    "entidad_San Luis Potosí": "24",
    "entidad_Sinaloa": "25",
    "entidad_Sonora": "26",
    "entidad_Tabasco": "27",
    "entidad_Tamaulipas": "28",
    "entidad_Tlaxcala": "29",
    "entidad_Veracruz": "30",
    "entidad_Yucatán": "31",
    "entidad_Zacatecas": "32",
    "cultivo_Resumen cultivos": "0",
    "cultivo_Agave": "6",
    "cultivo_Aguacate": "7",
    "cultivo_Ajo": "8",
    "cultivo_Ajonjolí": "9",
    "cultivo_Alfalfa": "15",
    "cultivo_Algodón hueso": "17",
    "cultivo_Amaranto": "27",
    "cultivo_Arroz palay": "36",
    "cultivo_Avena forrajera en verde": "42",
    "cultivo_Avena grano": "43",
    "cultivo_Berenjena": "53",
    "cultivo_Brócoli": "57",
    "cultivo_Cacao": "60",
    "cultivo_Café cereza": "61",
    "cultivo_Calabacita": "63",
    "cultivo_Caña de azúcar": "70",
    "cultivo_Cártamo": "74",
    "cultivo_Cebada grano": "77",
    "cultivo_Cebolla": "78",
    "cultivo_Chile verde": "89",
    "cultivo_Coliflor": "105",
    "cultivo_Copra": "110",
    "cultivo_Crisantemo": "112",
    "cultivo_Durazno": "122",
    "cultivo_Elote": "128",
    "cultivo_Espárrago": "132",
    "cultivo_Frambuesa": "149",
    "cultivo_Fresa": "150",
    "cultivo_Frijol": "152",
    "cultivo_Garbanzo grano": "159",
    "cultivo_Gerbera": "163",
    "cultivo_Gladiola": "168",
    "cultivo_Guayaba": "175",
    "cultivo_Lechuga": "206",
    "cultivo_Limón": "212",
    "cultivo_Maguey pulquero": "222",
    "cultivo_Maíz forrajero en verde": "224",
    "cultivo_Maíz grano": "225",
    "cultivo_Mango": "230",
    "cultivo_Manzana": "234",
    "cultivo_Melón": "241",
    "cultivo_Naranja": "259",
    "cultivo_Nopalitos": "268",
    "cultivo_Nuez": "272",
    "cultivo_Papa": "283",
    "cultivo_Papaya": "285",
    "cultivo_Pepino": "289",
    "cultivo_Pera": "290",
    "cultivo_Piña": "296",
    "cultivo_Plátano": "304",
    "cultivo_Rosa": "320",
    "cultivo_Sandía": "326",
    "cultivo_Sorgo forrajero en verde": "373",
    "cultivo_Sorgo grano": "374",
    "cultivo_Soya": "375",
    "cultivo_Tabaco": "379",
    "cultivo_Tomate rojo (jitomate)": "389",
    "cultivo_Tomate verde": "390",
    "cultivo_Toronja (pomelo)": "392",
    "cultivo_Trigo grano": "395",
    "cultivo_Tuna": "400",
    "cultivo_Uva": "401",
    "cultivo_Zanahoria": "416",
    "cultivo_Zarzamora": "420",
    "mesagric_Diciembre": "12",
    "mesagric_Noviembre": "11",
    "mesagric_Octubre": "10",
    "mesagric_Septiembre": "9",
    "distrito_Todos": "0",
    "distrito_Aguascalientes": "1",
    "municipio_Todos": "0",
    "municipio_Aguascalientes": "1",
    "municipio_Asientos": "2",
    "municipio_Calvillo": "3",
    "municipio_Cosío": "4",
    "municipio_El Llano": "10",
    "municipio_Jesús María": "48",
    "municipio_Pabellón de Arteaga": "6",
    "municipio_Rincón de Romos": "7",
    "municipio_San Francisco de Los Romo": "11",
    "municipio_San José de Gracia": "8",
    "municipio_Tepezalá": "9",
    "distrito_Ensenada": "2",
    "distrito_Río Colorado": "3",
    "municipio_Ensenada": "1",
    "municipio_Mexicali": "2",
    "municipio_Playas de Rosarito": "5",
    "municipio_San Felipe": "65",
    "municipio_San Quintín": "6",
    "municipio_Tecate": "3",
    "municipio_Tijuana": "4",
    "distrito_Comondú": "5",
    "distrito_La Paz": "6",
    "distrito_Los Cabos": "7",
    "distrito_Mulegé": "4",
    "municipio_Comondú": "1",
    "municipio_La Paz": "70",
    "municipio_Loreto": "24",
    "municipio_Los Cabos": "8",
    "municipio_Mulegé": "2",
    "distrito_Campeche": "9",
    "distrito_Champotón": "10",
    "distrito_Escárcega": "11",
    "distrito_Hecelchakán": "8",
    "municipio_Calakmul": "10",
    "municipio_Calkiní": "1",
    "municipio_Campeche": "2",
    "municipio_Candelaria": "11",
    "municipio_Carmen": "3",
    "municipio_Champotón": "4",
    "municipio_Dzitbalché": "13",
    "municipio_Escárcega": "9",
    "municipio_Hecelchakán": "5",
    "municipio_Hopelchén": "6",
    "municipio_Palizada": "7",
    "municipio_Seybaplaya": "12",
    "municipio_Tenabo": "8",
    "distrito_Acuña": "12",
    "distrito_Frontera": "14",
    "distrito_Laguna-Coahuila": "192",
    "distrito_Sabinas": "13",
    "distrito_Saltillo": "15",
    "municipio_Abasolo": "1",
    "municipio_Acuña": "2",
    "municipio_Allende": "4",
    "municipio_Arteaga": "10",
    "municipio_Candela": "5",
    "municipio_Castaños": "6",
    "municipio_Cuatro Ciénegas": "7",
    "municipio_Escobedo": "8",
    "municipio_Francisco I. Madero": "23",
    "municipio_Frontera": "10",
    "municipio_General Cepeda": "11",
    "municipio_Guerrero": "14",
    "municipio_Hidalgo": "16",
    "municipio_Jiménez": "18",
    "municipio_Juárez": "31",
    "municipio_Lamadrid": "16",
    "municipio_Matamoros": "22",
    "municipio_Monclova": "18",
    "municipio_Morelos": "32",
    "municipio_Múzquiz": "20",
    "municipio_Nadadores": "21",
    "municipio_Nava": "22",
    "municipio_Ocampo": "29",
    "municipio_Parras": "24",
    "municipio_Piedras Negras": "25",
    "municipio_Progreso": "59",
    "municipio_Ramos Arizpe": "27",
    "municipio_Sabinas": "28",
    "municipio_Sacramento": "29",
    "municipio_Saltillo": "30",
    "municipio_San Buenaventura": "31",
    "municipio_San Juan de Sabinas": "32",
    "municipio_San Pedro": "33",
    "municipio_Sierra Mojada": "34",
    "municipio_Torreón": "35",
    "municipio_Viesca": "36",
    "municipio_Villa Unión": "37",
    "municipio_Zaragoza": "199",
    "distrito_Colima": "16",
    "distrito_Tecomán": "17",
    "municipio_Armería": "1",
    "municipio_Colima": "2",
    "municipio_Comala": "3",
    "municipio_Coquimatlán": "4",
    "municipio_Cuauhtémoc": "8",
    "municipio_Ixtlahuacán": "6",
    "municipio_Manzanillo": "7",
    "municipio_Minatitlán": "108",
    "municipio_Tecomán": "9",
    "municipio_Villa de Álvarez": "10",
    "distrito_Comitán": "20",
    "distrito_Motozintla": "24",
    "distrito_Palenque": "23",
    "distrito_Pichucalco": "22",
    "distrito_San Cristóbal de las Casas": "19",
    "distrito_Selva Lacandona": "27",
    "distrito_Tapachula": "25",
    "distrito_Tonalá": "26",
    "distrito_Tuxtla Gutiérrez": "18",
    "distrito_Villa Flores": "21",
    "municipio_Acacoyagua": "1",
    "municipio_Acala": "2",
    "municipio_Acapetahua": "3",
    "municipio_Aldama": "2",
    "municipio_Altamirano": "4",
    "municipio_Amatán": "5",
    "municipio_Amatenango de La Frontera": "6",
    "municipio_Amatenango del Valle": "7",
    "municipio_Angel Albino Corzo": "8",
    "municipio_Arriaga": "9",
    "municipio_Bejucal de Ocampo": "10",
    "municipio_Bella Vista": "11",
    "municipio_Benemérito de Las Américas": "114",
    "municipio_Berriozábal": "12",
    "municipio_Bochil": "13",
    "municipio_Cacahoatán": "15",
    "municipio_Capitán Luis Ángel Vidal": "120",
    "municipio_Catazajá": "16",
    "municipio_Chalchihuitán": "22",
    "municipio_Chamula": "23",
    "municipio_Chanal": "24",
    "municipio_Chapultenango": "25",
    "municipio_Chenalhó": "26",
    "municipio_Chiapa de Corzo": "27",
    "municipio_Chiapilla": "28",
    "municipio_Chicoasén": "29",
    "municipio_Chicomuselo": "30",
    "municipio_Chilón": "31",
    "municipio_Cintalapa": "17",
    "municipio_Coapilla": "18",
    "municipio_Comitán de Domínguez": "19",
    "municipio_Copainalá": "21",
    "municipio_El Bosque": "14",
    "municipio_El Parral": "122",
    "municipio_El Porvenir": "70",
    "municipio_Emiliano Zapata": "65",
    "municipio_Escuintla": "32",
    "municipio_Francisco León": "33",
    "municipio_Frontera Comalapa": "34",
    "municipio_Frontera Hidalgo": "35",
    "municipio_Honduras de la Sierra": "125",
    "municipio_Huehuetán": "37",
    "municipio_Huitiupán": "39",
    "municipio_Huixtán": "38",
    "municipio_Huixtla": "40",
    "municipio_Ixhuatán": "42",
    "municipio_Ixtacomitán": "43",
    "municipio_Ixtapa": "44",
    "municipio_Ixtapangajoya": "45",
    "municipio_Jiquipilas": "46",
    "municipio_Jitotol": "47",
    "municipio_La Concordia": "20",
    "municipio_La Grandeza": "36",
    "municipio_La Independencia": "41",
    "municipio_La Libertad": "50",
    "municipio_La Trinitaria": "99",
    "municipio_Larráinzar": "49",
    "municipio_Las Margaritas": "52",
    "municipio_Las Rosas": "75",
    "municipio_Mapastepec": "51",
    "municipio_Maravilla Tenejapa": "115",
    "municipio_Marqués de Comillas": "116",
    "municipio_Mazapa de Madero": "53",
    "municipio_Mazatán": "37",
    "municipio_Metapa": "55",
    "municipio_Mezcalapa": "124",
    "municipio_Mitontic": "56",
    "municipio_Montecristo de Guerrero": "117",
    "municipio_Motozintla": "57",
    "municipio_Nicolás Ruíz": "58",
    "municipio_Ocosingo": "59",
    "municipio_Ocotepec": "105",
    "municipio_Ocozocoautla de Espinosa": "61",
    "municipio_Ostuacán": "62",
    "municipio_Osumacinta": "63",
    "municipio_Oxchuc": "64",
    "municipio_Palenque": "65",
    "municipio_Pantelhó": "66",
    "municipio_Pantepec": "111",
    "municipio_Pichucalco": "68",
    "municipio_Pijijiapan": "69",
    "municipio_Pueblo Nuevo Solistahuacán": "72",
    "municipio_Rayón": "50",
    "municipio_Reforma": "74",
    "municipio_Rincón Chamula San Pedro": "121",
    "municipio_Sabanilla": "76",
    "municipio_Salto de Agua": "77",
    "municipio_San Andrés Duraznal": "118",
    "municipio_San Cristóbal de Las Casas": "78",
    "municipio_San Fernando": "35",
    "municipio_San Juan Cancuc": "112",
    "municipio_San Lucas": "77",
    "municipio_Santiago El Pinar": "119",
    "municipio_Siltepec": "80",
    "municipio_Simojovel": "81",
    "municipio_Sitalá": "82",
    "municipio_Socoltenango": "83",
    "municipio_Solosuchiapa": "84",
    "municipio_Soyaló": "85",
    "municipio_Suchiapa": "86",
    "municipio_Suchiate": "87",
    "municipio_Sunuapa": "88",
    "municipio_Tapachula": "89",
    "municipio_Tapalapa": "90",
    "municipio_Tapilula": "91",
    "municipio_Tecpatán": "92",
    "municipio_Tenejapa": "93",
    "municipio_Teopisca": "94",
    "municipio_Tila": "96",
    "municipio_Tonalá": "101",
    "municipio_Totolapa": "98",
    "municipio_Tumbalá": "100",
    "municipio_Tuxtla Chico": "102",
    "municipio_Tuxtla Gutiérrez": "101",
    "municipio_Tuzantán": "103",
    "municipio_Tzimol": "104",
    "municipio_Unión Juárez": "105",
    "municipio_Venustiano Carranza": "194",
    "municipio_Villa Comaltitlán": "71",
    "municipio_Villa Corzo": "107",
    "municipio_Villaflores": "108",
    "municipio_Yajalón": "109",
    "municipio_Zinacantán": "111",
    "distrito_Bajo Río Conchos": "36",
    "distrito_Balleza": "38",
    "distrito_Buenaventura": "29",
    "distrito_Casas Grandes": "28",
    "distrito_Chihuahua": "35",
    "distrito_Cuauhtémoc": "33",
    "distrito_Delicias": "40",
    "distrito_El Carmen": "30",
    "distrito_Madera": "32",
    "distrito_Papigochi": "34",
    "distrito_Parral": "39",
    "distrito_Río Florido": "41",
    "distrito_San Juanito": "37",
    "distrito_Valle de Juárez": "31",
    "municipio_Ahumada": "1",
    "municipio_Aquiles Serdán": "4",
    "municipio_Ascensión": "5",
    "municipio_Bachíniva": "6",
    "municipio_Balleza": "7",
    "municipio_Batopilas": "8",
    "municipio_Bocoyna": "9",
    "municipio_Buenaventura": "10",
    "municipio_Camargo": "7",
    "municipio_Carichí": "12",
    "municipio_Casas Grandes": "13",
    "municipio_Chihuahua": "19",
    "municipio_Chínipas": "20",
    "municipio_Coronado": "14",
    "municipio_Coyame del Sotol": "15",
    "municipio_Cusihuiriachi": "18",
    "municipio_Delicias": "21",
    "municipio_Dr. Belisario Domínguez": "22",
    "municipio_El Tule": "64",
    "municipio_Galeana": "17",
    "municipio_Gómez Farías": "11",
    "municipio_Gran Morelos": "26",
    "municipio_Guachochi": "27",
    "municipio_Guadalupe": "17",
    "municipio_Guadalupe Y Calvo": "29",
    "municipio_Guazapares": "30",
    "municipio_Hidalgo del Parral": "32",
    "municipio_Huejotitán": "33",
    "municipio_Ignacio Zaragoza": "34",
    "municipio_Janos": "35",
    "municipio_Julimes": "38",
    "municipio_La Cruz": "16",
    "municipio_López": "39",
    "municipio_Madera": "40",
    "municipio_Maguarichi": "41",
    "municipio_Manuel Benavides": "42",
    "municipio_Matachí": "43",
    "municipio_Meoqui": "45",
    "municipio_Moris": "47",
    "municipio_Namiquipa": "48",
    "municipio_Nonoava": "49",
    "municipio_Nuevo Casas Grandes": "50",
    "municipio_Ojinaga": "52",
    "municipio_Praxedis G. Guerrero": "53",
    "municipio_Riva Palacio": "54",
    "municipio_Rosales": "55",
    "municipio_Rosario": "51",
    "municipio_San Francisco de Borja": "57",
    "municipio_San Francisco de Conchos": "58",
    "municipio_San Francisco del Oro": "59",
    "municipio_Santa Bárbara": "60",
    "municipio_Santa Isabel": "24",
    "municipio_Satevó": "61",
    "municipio_Saucillo": "62",
    "municipio_Temósachic": "63",
    "municipio_Urique": "65",
    "municipio_Uruachi": "66",
    "municipio_Valle de Zaragoza": "67",
    "distrito_": "-1",
    "distrito_Xochimilco": "42",
    "municipio_Álvaro Obregón": "3",
    "municipio_Azcapotzalco": "2",
    "municipio_Benito Juárez": "4",
    "municipio_Coyoacán": "3",
    "municipio_Cuajimalpa de Morelos": "4",
    "municipio_Gustavo A. Madero": "5",
    "municipio_Iztacalco": "6",
    "municipio_Iztapalapa": "7",
    "municipio_La Magdalena Contreras": "8",
    "municipio_Miguel Hidalgo": "16",
    "municipio_Milpa Alta": "9",
    "municipio_Tláhuac": "11",
    "municipio_Tlalpan": "12",
    "municipio_Xochimilco": "13",
    "distrito_Durango": "43",
    "distrito_El Salto": "44",
    "distrito_Guadalupe Victoria": "45",
    "distrito_Laguna-Durango": "191",
    "distrito_Santiago Papasquiaro": "47",
    "distrito_Villa Ocampo": "46",
    "municipio_Canatlán": "1",
    "municipio_Canelas": "2",
    "municipio_Coneto de Comonfort": "3",
    "municipio_Cuencamé": "4",
    "municipio_Durango": "5",
    "municipio_El Oro": "64",
    "municipio_General Simón Bolívar": "6",
    "municipio_Gómez Palacio": "7",
    "municipio_Guadalupe Victoria": "67",
    "municipio_Guanaceví": "9",
    "municipio_Indé": "11",
    "municipio_Lerdo": "12",
    "municipio_Mapimí": "13",
    "municipio_Mezquital": "14",
    "municipio_Nazas": "15",
    "municipio_Nombre de Dios": "16",
    "municipio_Nuevo Ideal": "39",
    "municipio_Otáez": "19",
    "municipio_Pánuco de Coronado": "20",
    "municipio_Peñón Blanco": "21",
    "municipio_Poanas": "22",
    "municipio_Pueblo Nuevo": "24",
    "municipio_Rodeo": "24",
    "municipio_San Bernardo": "25",
    "municipio_San Dimas": "26",
    "municipio_San Juan de Guadalupe": "27",
    "municipio_San Juan del Río": "16",
    "municipio_San Luis del Cordero": "29",
    "municipio_San Pedro del Gallo": "30",
    "municipio_Santa Clara": "31",
    "municipio_Santiago Papasquiaro": "32",
    "municipio_Súchil": "33",
    "municipio_Tamazula": "34",
    "municipio_Tepehuanes": "35",
    "municipio_Tlahualilo": "36",
    "municipio_Topia": "37",
    "municipio_Vicente Guerrero": "195",
    "distrito_Celaya": "51",
    "distrito_Cortazar": "52",
    "distrito_Dolores Hidalgo": "48",
    "distrito_León": "50",
    "distrito_San Luis de la Paz": "49",
    "municipio_Acámbaro": "2",
    "municipio_Apaseo El Alto": "4",
    "municipio_Apaseo El Grande": "5",
    "municipio_Atarjea": "6",
    "municipio_Celaya": "7",
    "municipio_Comonfort": "9",
    "municipio_Coroneo": "10",
    "municipio_Cortazar": "11",
    "municipio_Cuerámaro": "12",
    "municipio_Doctor Mora": "13",
    "municipio_Dolores Hidalgo Cuna de la Independencia Nacional": "14",
    "municipio_Guanajuato": "15",
    "municipio_Huanímaro": "16",
    "municipio_Irapuato": "17",
    "municipio_Jaral del Progreso": "18",
    "municipio_Jerécuaro": "19",
    "municipio_León": "20",
    "municipio_Manuel Doblado": "8",
    "municipio_Moroleón": "21",
    "municipio_Pénjamo": "23",
    "municipio_Purísima del Rincón": "25",
    "municipio_Romita": "26",
    "municipio_Salamanca": "27",
    "municipio_Salvatierra": "28",
    "municipio_San Diego de La Unión": "29",
    "municipio_San Francisco del Rincón": "31",
    "municipio_San José Iturbide": "32",
    "municipio_San Luis de La Paz": "33",
    "municipio_San Miguel de Allende": "3",
    "municipio_Santa Catarina": "31",
    "municipio_Santa Cruz de Juventino Rosas": "35",
    "municipio_Santiago Maravatío": "36",
    "municipio_Silao de La Victoria": "37",
    "municipio_Tarandacuao": "38",
    "municipio_Tarimoro": "39",
    "municipio_Tierra Blanca": "174",
    "municipio_Uriangato": "41",
    "municipio_Valle de Santiago": "42",
    "municipio_Victoria": "41",
    "municipio_Villagrán": "42",
    "municipio_Xichú": "45",
    "municipio_Yuriria": "46",
    "distrito_Altamirano": "55",
    "distrito_Atoyac": "53",
    "distrito_Chilpancingo": "56",
    "distrito_Iguala": "58",
    "distrito_Las Vigas": "54",
    "distrito_Tlapa": "57",
    "municipio_Acapulco de Juárez": "1",
    "municipio_Acatepec": "76",
    "municipio_Ahuacuotzingo": "2",
    "municipio_Ajuchitlán del Progreso": "3",
    "municipio_Alcozauca de Guerrero": "4",
    "municipio_Alpoyeca": "5",
    "municipio_Apaxtla": "6",
    "municipio_Arcelia": "7",
    "municipio_Atenango del Río": "8",
    "municipio_Atlamajalcingo del Monte": "9",
    "municipio_Atlixtac": "10",
    "municipio_Atoyac de Álvarez": "11",
    "municipio_Ayutla de Los Libres": "12",
    "municipio_Azoyú": "13",
    "municipio_Buenavista de Cuéllar": "15",
    "municipio_Chilapa de Álvarez": "28",
    "municipio_Chilpancingo de Los Bravo": "29",
    "municipio_Coahuayutla de José María Izazaga": "16",
    "municipio_Cochoapa El Grande": "78",
    "municipio_Cocula": "24",
    "municipio_Copala": "18",
    "municipio_Copalillo": "19",
    "municipio_Copanatoyac": "20",
    "municipio_Coyuca de Benítez": "21",
    "municipio_Coyuca de Catalán": "22",
    "municipio_Cuajinicuilapa": "23",
    "municipio_Cualác": "24",
    "municipio_Cuautepec": "25",
    "municipio_Cuetzala del Progreso": "26",
    "municipio_Cutzamala de Pinzón": "27",
    "municipio_Eduardo Neri": "75",
    "municipio_Florencio Villarreal": "30",
    "municipio_General Canuto A. Neri": "31",
    "municipio_General Heliodoro Castillo": "32",
    "municipio_Huamuxtitlán": "33",
    "municipio_Huitzuco de Los Figueroa": "34",
    "municipio_Iguala de La Independencia": "35",
    "municipio_Igualapa": "36",
    "municipio_Iliatenco": "81",
    "municipio_Ixcateopan de Cuauhtémoc": "37",
    "municipio_José Joaquín de Herrera": "79",
    "municipio_Juan R. Escudero": "39",
    "municipio_Juchitán": "80",
    "municipio_La Unión de Isidoro Montes de Oca": "68",
    "municipio_Leonardo Bravo": "40",
    "municipio_Malinaltepec": "41",
    "municipio_Marquelia": "77",
    "municipio_Mártir de Cuilapan": "42",
    "municipio_Metlatónoc": "43",
    "municipio_Mochitlán": "44",
    "municipio_Olinalá": "45",
    "municipio_Ometepec": "46",
    "municipio_Pedro Ascencio Alquisiras": "47",
    "municipio_Petatlán": "48",
    "municipio_Pilcaya": "49",
    "municipio_Pungarabato": "50",
    "municipio_Quechultenango": "51",
    "municipio_San Luis Acatlán": "52",
    "municipio_San Marcos": "75",
    "municipio_San Miguel Totolapan": "54",
    "municipio_Taxco de Alarcón": "55",
    "municipio_Tecoanapa": "56",
    "municipio_Técpan de Galeana": "57",
    "municipio_Teloloapan": "58",
    "municipio_Tepecoacuilco de Trujano": "59",
    "municipio_Tetipac": "60",
    "municipio_Tixtla de Guerrero": "61",
    "municipio_Tlacoachistlahuaca": "62",
    "municipio_Tlacoapa": "63",
    "municipio_Tlalchapa": "64",
    "municipio_Tlalixtaquilla de Maldonado": "65",
    "municipio_Tlapa de Comonfort": "66",
    "municipio_Tlapehuala": "67",
    "municipio_Xalpatláhuac": "69",
    "municipio_Xochihuehuetlán": "70",
    "municipio_Xochistlahuaca": "71",
    "municipio_Zapotitlán Tablas": "72",
    "municipio_Zihuatanejo de Azueta": "38",
    "municipio_Zirándaro": "73",
    "municipio_Zitlala": "74",
    "distrito_Huejutla": "59",
    "distrito_Huichapan": "64",
    "distrito_Mixquiahuala": "63",
    "distrito_Pachuca": "62",
    "distrito_Tulancingo": "61",
    "distrito_Zacualtipán": "60",
    "municipio_Acatlán": "2",
    "municipio_Acaxochitlán": "2",
    "municipio_Actopan": "4",
    "municipio_Agua Blanca de Iturbide": "4",
    "municipio_Ajacuba": "5",
    "municipio_Alfajayucan": "6",
    "municipio_Almoloya": "7",
    "municipio_Apan": "8",
    "municipio_Atitalaquia": "10",
    "municipio_Atlapexco": "11",
    "municipio_Atotonilco de Tula": "13",
    "municipio_Atotonilco El Grande": "12",
    "municipio_Calnali": "14",
    "municipio_Cardonal": "15",
    "municipio_Chapantongo": "17",
    "municipio_Chapulhuacán": "18",
    "municipio_Chilcuautla": "19",
    "municipio_Cuautepec de Hinojosa": "16",
    "municipio_El Arenal": "9",
    "municipio_Eloxochitlán": "61",
    "municipio_Epazoyucan": "22",
    "municipio_Huasca de Ocampo": "24",
    "municipio_Huautla": "25",
    "municipio_Huazalingo": "26",
    "municipio_Huehuetla": "72",
    "municipio_Huejutla de Reyes": "28",
    "municipio_Huichapan": "29",
    "municipio_Ixmiquilpan": "30",
    "municipio_Jacala de Ledezma": "31",
    "municipio_Jaltocán": "32",
    "municipio_Juárez Hidalgo": "33",
    "municipio_La Misión": "40",
    "municipio_Lolotla": "34",
    "municipio_Metepec": "54",
    "municipio_Metztitlán": "37",
    "municipio_Mineral de La Reforma": "51",
    "municipio_Mineral del Chico": "38",
    "municipio_Mineral del Monte": "39",
    "municipio_Mixquiahuala de Juárez": "41",
    "municipio_Molango de Escamilla": "42",
    "municipio_Nicolás Flores": "43",
    "municipio_Nopala de Villagrán": "44",
    "municipio_Omitlán de Juárez": "45",
    "municipio_Pachuca de Soto": "48",
    "municipio_Pacula": "47",
    "municipio_Pisaflores": "49",
    "municipio_Progreso de Obregón": "50",
    "municipio_San Agustín Metzquititlán": "36",
    "municipio_San Agustín Tlaxiaca": "52",
    "municipio_San Bartolo Tutotepec": "53",
    "municipio_San Felipe Orizatlán": "46",
    "municipio_San Salvador": "54",
    "municipio_Santiago de Anaya": "55",
    "municipio_Santiago Tulantepec de Lugo Guerrero": "56",
    "municipio_Singuilucan": "57",
    "municipio_Tasquillo": "58",
    "municipio_Tecozautla": "59",
    "municipio_Tenango de Doria": "60",
    "municipio_Tepeapulco": "61",
    "municipio_Tepehuacán de Guerrero": "62",
    "municipio_Tepeji del Río de Ocampo": "63",
    "municipio_Tepetitlán": "64",
    "municipio_Tetepango": "65",
    "municipio_Tezontepec de Aldama": "67",
    "municipio_Tianguistengo": "68",
    "municipio_Tizayuca": "69",
    "municipio_Tlahuelilpan": "70",
    "municipio_Tlahuiltepa": "71",
    "municipio_Tlanalapa": "72",
    "municipio_Tlanchinol": "73",
    "municipio_Tlaxcoapan": "74",
    "municipio_Tolcayuca": "75",
    "municipio_Tula de Allende": "76",
    "municipio_Tulancingo de Bravo": "77",
    "municipio_Villa de Tezontepec": "66",
    "municipio_Xochiatipan": "78",
    "municipio_Xochicoatlán": "79",
    "municipio_Yahualica": "80",
    "municipio_Zacualtipán de Ángeles": "81",
    "municipio_Zapotlán de Juárez": "82",
    "municipio_Zempoala": "83",
    "municipio_Zimapán": "84",
    "distrito_Ameca": "67",
    "distrito_Ciudad Guzmán": "71",
    "distrito_Colotlán": "72",
    "distrito_El Grullo": "69",
    "distrito_La Barca": "70",
    "distrito_Lagos de Moreno": "66",
    "distrito_Tomatlán": "68",
    "distrito_Zapopan": "65",
    "municipio_Acatic": "1",
    "municipio_Acatlán de Juárez": "2",
    "municipio_Ahualulco de Mercado": "3",
    "municipio_Amacueca": "4",
    "municipio_Amatitán": "5",
    "municipio_Ameca": "6",
    "municipio_Arandas": "8",
    "municipio_Atemajac de Brizuela": "10",
    "municipio_Atengo": "11",
    "municipio_Atenguillo": "12",
    "municipio_Atotonilco El Alto": "13",
    "municipio_Atoyac": "21",
    "municipio_Autlán de Navarro": "15",
    "municipio_Ayotlán": "16",
    "municipio_Ayutla": "17",
    "municipio_Bolaños": "19",
    "municipio_Cabo Corrientes": "20",
    "municipio_Cañadas de Obregón": "117",
    "municipio_Casimiro Castillo": "21",
    "municipio_Chapala": "30",
    "municipio_Chimaltitán": "31",
    "municipio_Chiquilistlán": "32",
    "municipio_Cihuatlán": "22",
    "municipio_Colotlán": "25",
    "municipio_Concepción de Buenos Aires": "26",
    "municipio_Cuautitlán de García Barragán": "27",
    "municipio_Cuautla": "6",
    "municipio_Cuquío": "29",
    "municipio_Degollado": "33",
    "municipio_Ejutla": "34",
    "municipio_El Grullo": "37",
    "municipio_El Limón": "54",
    "municipio_El Salto": "70",
    "municipio_Encarnación de Díaz": "35",
    "municipio_Etzatlán": "36",
    "municipio_Guachinango": "38",
    "municipio_Guadalajara": "39",
    "municipio_Hostotipaquillo": "40",
    "municipio_Huejúcar": "41",
    "municipio_Huejuquilla El Alto": "42",
    "municipio_Ixtlahuacán de Los Membrillos": "44",
    "municipio_Ixtlahuacán del Río": "45",
    "municipio_Jalostotitlán": "46",
    "municipio_Jamay": "47",
    "municipio_Jilotlán de Los Dolores": "49",
    "municipio_Jocotepec": "50",
    "municipio_Juanacatlán": "51",
    "municipio_Juchitlán": "52",
    "municipio_La Barca": "18",
    "municipio_La Huerta": "43",
    "municipio_La Manzanilla de la Paz": "57",
    "municipio_Lagos de Moreno": "53",
    "municipio_Magdalena": "98",
    "municipio_Mascota": "58",
    "municipio_Mazamitla": "59",
    "municipio_Mexticacán": "60",
    "municipio_Mezquitic": "61",
    "municipio_Mixtlán": "62",
    "municipio_Ocotlán": "63",
    "municipio_Ojuelos de Jalisco": "64",
    "municipio_Pihuamo": "65",
    "municipio_Poncitlán": "66",
    "municipio_Puerto Vallarta": "67",
    "municipio_Quitupan": "69",
    "municipio_San Cristóbal de La Barranca": "71",
    "municipio_San Diego de Alejandría": "72",
    "municipio_San Gabriel": "113",
    "municipio_San Ignacio Cerro Gordo": "125",
    "municipio_San Juan de Los Lagos": "73",
    "municipio_San Juanito de Escobedo": "7",
    "municipio_San Julián": "74",
    "municipio_San Martín de Bolaños": "76",
    "municipio_San Martín Hidalgo": "77",
    "municipio_San Miguel El Alto": "78",
    "municipio_San Pedro Tlaquepaque": "98",
    "municipio_San Sebastián del Oeste": "80",
    "municipio_Santa María de Los Ángeles": "81",
    "municipio_Santa María del Oro": "14",
    "municipio_Sayula": "82",
    "municipio_Tala": "83",
    "municipio_Talpa de Allende": "84",
    "municipio_Tamazula de Gordiano": "85",
    "municipio_Tapalpa": "86",
    "municipio_Tecalitlán": "87",
    "municipio_Techaluta de Montenegro": "89",
    "municipio_Tecolotlán": "88",
    "municipio_Tenamaxtlán": "90",
    "municipio_Teocaltiche": "91",
    "municipio_Teocuitatlán de Corona": "92",
    "municipio_Tepatitlán de Morelos": "93",
    "municipio_Tequila": "168",
    "municipio_Teuchitlán": "95",
    "municipio_Tizapán El Alto": "96",
    "municipio_Tlajomulco de Zúñiga": "97",
    "municipio_Tolimán": "18",
    "municipio_Tomatlán": "186",
    "municipio_Tonaya": "102",
    "municipio_Tonila": "103",
    "municipio_Totatiche": "104",
    "municipio_Tototlán": "105",
    "municipio_Tuxcacuesco": "106",
    "municipio_Tuxcueca": "107",
    "municipio_Tuxpan": "189",
    "municipio_Unión de San Antonio": "109",
    "municipio_Unión de Tula": "110",
    "municipio_Valle de Guadalupe": "111",
    "municipio_Valle de Juárez": "112",
    "municipio_Villa Corona": "114",
    "municipio_Villa Guerrero": "113",
    "municipio_Villa Hidalgo": "54",
    "municipio_Villa Purificación": "68",
    "municipio_Yahualica de González Gallo": "118",
    "municipio_Zacoalco de Torres": "119",
    "municipio_Zapopan": "120",
    "municipio_Zapotiltic": "121",
    "municipio_Zapotitlán de Vadillo": "122",
    "municipio_Zapotlán del Rey": "123",
    "municipio_Zapotlán el Grande": "23",
    "municipio_Zapotlanejo": "124",
    "distrito_Atlacomulco": "77",
    "distrito_Coatepec Harinas": "78",
    "distrito_Jilotepec": "80",
    "distrito_Tejupilco": "76",
    "distrito_Texcoco": "75",
    "distrito_Toluca": "73",
    "distrito_Valle de Bravo": "79",
    "distrito_Zumpango": "74",
    "municipio_Acambay de Ruíz Castañeda": "1",
    "municipio_Acolman": "2",
    "municipio_Aculco": "3",
    "municipio_Almoloya de Alquisiras": "4",
    "municipio_Almoloya de Juárez": "5",
    "municipio_Almoloya del Río": "6",
    "municipio_Amanalco": "7",
    "municipio_Amatepec": "8",
    "municipio_Amecameca": "9",
    "municipio_Apaxco": "10",
    "municipio_Atenco": "11",
    "municipio_Atizapán": "12",
    "municipio_Atizapán de Zaragoza": "13",
    "municipio_Atlacomulco": "14",
    "municipio_Atlautla": "15",
    "municipio_Axapusco": "16",
    "municipio_Ayapango": "17",
    "municipio_Calimaya": "18",
    "municipio_Capulhuac": "19",
    "municipio_Chalco": "25",
    "municipio_Chapa de Mota": "26",
    "municipio_Chapultepec": "27",
    "municipio_Chiautla": "47",
    "municipio_Chicoloapan": "29",
    "municipio_Chiconcuac": "30",
    "municipio_Chimalhuacán": "31",
    "municipio_Coacalco de Berriozábal": "20",
    "municipio_Coatepec Harinas": "21",
    "municipio_Cocotitlán": "22",
    "municipio_Coyotepec": "37",
    "municipio_Cuautitlán": "24",
    "municipio_Cuautitlán Izcalli": "121",
    "municipio_Donato Guerra": "32",
    "municipio_Ecatepec de Morelos": "33",
    "municipio_Ecatzingo": "34",
    "municipio_Huehuetoca": "35",
    "municipio_Hueypoxtla": "36",
    "municipio_Huixquilucan": "37",
    "municipio_Isidro Fabela": "38",
    "municipio_Ixtapaluca": "39",
    "municipio_Ixtapan de La Sal": "40",
    "municipio_Ixtapan del Oro": "41",
    "municipio_Ixtlahuaca": "42",
    "municipio_Jaltenco": "44",
    "municipio_Jilotepec": "93",
    "municipio_Jilotzingo": "46",
    "municipio_Jiquipilco": "47",
    "municipio_Jocotitlán": "48",
    "municipio_Joquicingo": "49",
    "municipio_Juchitepec": "50",
    "municipio_Lerma": "51",
    "municipio_Luvianos": "123",
    "municipio_Malinalco": "52",
    "municipio_Melchor Ocampo": "27",
    "municipio_Mexicaltzingo": "55",
    "municipio_Naucalpan de Juárez": "57",
    "municipio_Nextlalpan": "59",
    "municipio_Nezahualcóyotl": "58",
    "municipio_Nicolás Romero": "60",
    "municipio_Nopaltepec": "61",
    "municipio_Ocoyoacac": "62",
    "municipio_Ocuilan": "63",
    "municipio_Otumba": "65",
    "municipio_Otzoloapan": "66",
    "municipio_Otzolotepec": "67",
    "municipio_Ozumba": "68",
    "municipio_Papalotla": "69",
    "municipio_Polotitlán": "71",
    "municipio_San Antonio La Isla": "73",
    "municipio_San Felipe del Progreso": "74",
    "municipio_San José del Rincón": "124",
    "municipio_San Martín de Las Pirámides": "75",
    "municipio_San Mateo Atenco": "76",
    "municipio_San Simón de Guerrero": "77",
    "municipio_Santo Tomás": "78",
    "municipio_Soyaniquilpan de Juárez": "79",
    "municipio_Sultepec": "80",
    "municipio_Tecámac": "81",
    "municipio_Tejupilco": "82",
    "municipio_Temamatla": "83",
    "municipio_Temascalapa": "84",
    "municipio_Temascalcingo": "85",
    "municipio_Temascaltepec": "86",
    "municipio_Temoaya": "87",
    "municipio_Tenancingo": "27",
    "municipio_Tenango del Aire": "89",
    "municipio_Tenango del Valle": "90",
    "municipio_Teoloyucan": "91",
    "municipio_Teotihuacán": "92",
    "municipio_Tepetlaoxtoc": "93",
    "municipio_Tepetlixpa": "94",
    "municipio_Tepotzotlán": "95",
    "municipio_Tequixquiac": "96",
    "municipio_Texcaltitlán": "97",
    "municipio_Texcalyacac": "98",
    "municipio_Texcoco": "99",
    "municipio_Tezoyuca": "100",
    "municipio_Tianguistenco": "101",
    "municipio_Timilpan": "102",
    "municipio_Tlalmanalco": "103",
    "municipio_Tlalnepantla de Baz": "104",
    "municipio_Tlatlaya": "105",
    "municipio_Toluca": "106",
    "municipio_Tonanitla": "125",
    "municipio_Tonatico": "107",
    "municipio_Tultepec": "108",
    "municipio_Tultitlán": "109",
    "municipio_Valle de Bravo": "110",
    "municipio_Valle de Chalco Solidaridad": "122",
    "municipio_Villa de Allende": "111",
    "municipio_Villa del Carbón": "112",
    "municipio_Villa Victoria": "114",
    "municipio_Xalatlaco": "43",
    "municipio_Xonacatlán": "115",
    "municipio_Zacazonapan": "116",
    "municipio_Zacualpan": "198",
    "municipio_Zinacantepec": "118",
    "municipio_Zumpahuacán": "119",
    "municipio_Zumpango": "120",
    "distrito_Aguililla": "82",
    "distrito_Apatzingán": "85",
    "distrito_Coahuayana": "81",
    "distrito_Huacana": "84",
    "distrito_Huetamo": "92",
    "distrito_La Piedad": "89",
    "distrito_Lázaro Cárdenas": "83",
    "distrito_Morelia": "91",
    "distrito_Pátzcuaro": "90",
    "distrito_Sahuayo": "88",
    "distrito_Uruapan": "86",
    "distrito_Zamora": "87",
    "distrito_Zitácuaro": "93",
    "municipio_Acuitzio": "1",
    "municipio_Aguililla": "2",
    "municipio_Angamacutiro": "4",
    "municipio_Angangueo": "5",
    "municipio_Apatzingán": "6",
    "municipio_Aporo": "7",
    "municipio_Aquila": "18",
    "municipio_Ario": "9",
    "municipio_Briseñas": "11",
    "municipio_Buenavista": "12",
    "municipio_Carácuaro": "13",
    "municipio_Charapan": "21",
    "municipio_Charo": "22",
    "municipio_Chavinda": "23",
    "municipio_Cherán": "24",
    "municipio_Chilchota": "25",
    "municipio_Chinicuila": "26",
    "municipio_Chucándiro": "27",
    "municipio_Churintzio": "28",
    "municipio_Churumuco": "29",
    "municipio_Coahuayana": "14",
    "municipio_Coalcomán de Vázquez Pallares": "15",
    "municipio_Coeneo": "16",
    "municipio_Cojumatlán de Régules": "74",
    "municipio_Contepec": "17",
    "municipio_Copándaro": "18",
    "municipio_Cotija": "19",
    "municipio_Cuitzeo": "20",
    "municipio_Ecuandureo": "30",
    "municipio_Epitacio Huerta": "31",
    "municipio_Erongarícuaro": "32",
    "municipio_Gabriel Zamora": "33",
    "municipio_Huandacareo": "36",
    "municipio_Huaniqueo": "37",
    "municipio_Huetamo": "38",
    "municipio_Huiramba": "39",
    "municipio_Indaparapeo": "40",
    "municipio_Irimbo": "41",
    "municipio_Ixtlán": "42",
    "municipio_Jacona": "43",
    "municipio_Jiquilpan": "45",
    "municipio_José Sixto Verduzco": "113",
    "municipio_Jungapeo": "47",
    "municipio_La Huacana": "35",
    "municipio_La Piedad": "69",
    "municipio_Lagunillas": "19",
    "municipio_Lázaro Cárdenas": "47",
    "municipio_Los Reyes": "137",
    "municipio_Madero": "49",
    "municipio_Maravatío": "50",
    "municipio_Marcos Castellanos": "51",
    "municipio_Morelia": "53",
    "municipio_Múgica": "55",
    "municipio_Nahuatzen": "56",
    "municipio_Nocupétaro": "57",
    "municipio_Nuevo Parangaricutiro": "58",
    "municipio_Nuevo Urecho": "59",
    "municipio_Numarán": "60",
    "municipio_Pajacuarán": "62",
    "municipio_Panindícuaro": "63",
    "municipio_Paracho": "65",
    "municipio_Parácuaro": "64",
    "municipio_Pátzcuaro": "66",
    "municipio_Penjamillo": "67",
    "municipio_Peribán": "68",
    "municipio_Purépero": "70",
    "municipio_Puruándiro": "71",
    "municipio_Queréndaro": "72",
    "municipio_Quiroga": "73",
    "municipio_Sahuayo": "76",
    "municipio_Salvador Escalante": "79",
    "municipio_Santa Ana Maya": "78",
    "municipio_Senguio": "80",
    "municipio_Susupuato": "81",
    "municipio_Tacámbaro": "82",
    "municipio_Tancítaro": "83",
    "municipio_Tangamandapio": "84",
    "municipio_Tangancícuaro": "85",
    "municipio_Tanhuato": "86",
    "municipio_Taretan": "87",
    "municipio_Tarímbaro": "88",
    "municipio_Tepalcatepec": "89",
    "municipio_Tingambato": "90",
    "municipio_Tingüindín": "91",
    "municipio_Tiquicheo de Nicolás Romero": "92",
    "municipio_Tlalpujahua": "93",
    "municipio_Tlazazalca": "94",
    "municipio_Tocumbo": "95",
    "municipio_Tumbiscatío": "96",
    "municipio_Turicato": "97",
    "municipio_Tuzantla": "99",
    "municipio_Tzintzuntzan": "100",
    "municipio_Tzitzio": "101",
    "municipio_Uruapan": "102",
    "municipio_Villamar": "104",
    "municipio_Vista Hermosa": "105",
    "municipio_Yurécuaro": "106",
    "municipio_Zacapu": "107",
    "municipio_Zamora": "108",
    "municipio_Zináparo": "109",
    "municipio_Zinapécuaro": "110",
    "municipio_Ziracuaretiro": "111",
    "municipio_Zitácuaro": "112",
    "distrito_Zacatepec-Galeana": "94",
    "municipio_Amacuzac": "1",
    "municipio_Atlatlahucan": "2",
    "municipio_Axochiapan": "3",
    "municipio_Ayala": "4",
    "municipio_Coatetelco": "34",
    "municipio_Coatlán del Río": "5",
    "municipio_Cuernavaca": "7",
    "municipio_Hueyapan": "75",
    "municipio_Huitzilac": "9",
    "municipio_Jantetelco": "10",
    "municipio_Jiutepec": "11",
    "municipio_Jojutla": "12",
    "municipio_Jonacatepec": "13",
    "municipio_Mazatepec": "14",
    "municipio_Miacatlán": "15",
    "municipio_Ocuituco": "16",
    "municipio_Puente de Ixtla": "17",
    "municipio_Temixco": "18",
    "municipio_Temoac": "33",
    "municipio_Tepalcingo": "19",
    "municipio_Tepoztlán": "20",
    "municipio_Tetecala": "21",
    "municipio_Tetela del Volcán": "22",
    "municipio_Tlalnepantla": "23",
    "municipio_Tlaltizapán de Zapata": "24",
    "municipio_Tlaquiltenango": "25",
    "municipio_Tlayacapan": "26",
    "municipio_Totolapan": "27",
    "municipio_Xochitepec": "28",
    "municipio_Xoxocotla": "195",
    "municipio_Yautepec": "29",
    "municipio_Yecapixtla": "30",
    "municipio_Zacatepec": "31",
    "distrito_Acaponeta": "98",
    "distrito_Ahuacatlán": "97",
    "distrito_Compostela": "96",
    "distrito_Santiago Ixcuintla": "95",
    "distrito_Tepic": "99",
    "municipio_Acaponeta": "1",
    "municipio_Ahuacatlán": "6",
    "municipio_Amatlán de Cañas": "3",
    "municipio_Bahía de Banderas": "20",
    "municipio_Compostela": "4",
    "municipio_Del Nayar": "9",
    "municipio_Huajicori": "5",
    "municipio_Ixtlán del Río": "6",
    "municipio_Jala": "7",
    "municipio_La Yesca": "19",
    "municipio_Rosamorada": "10",
    "municipio_Ruíz": "11",
    "municipio_San Blas": "12",
    "municipio_San Pedro Lagunillas": "13",
    "municipio_Santiago Ixcuintla": "15",
    "municipio_Tecuala": "16",
    "municipio_Tepic": "17",
    "municipio_Xalisco": "8",
    "distrito_Anáhuac": "100",
    "distrito_Apodaca": "101",
    "distrito_Galeana": "103",
    "distrito_Montemorelos": "102",
    "municipio_Agualeguas": "2",
    "municipio_Anáhuac": "5",
    "municipio_Apodaca": "6",
    "municipio_Aramberri": "7",
    "municipio_Bustamante": "6",
    "municipio_Cadereyta Jiménez": "9",
    "municipio_Cerralvo": "11",
    "municipio_China": "13",
    "municipio_Ciénega de Flores": "12",
    "municipio_Doctor Arroyo": "14",
    "municipio_Doctor Coss": "15",
    "municipio_Doctor González": "16",
    "municipio_El Carmen": "10",
    "municipio_García": "18",
    "municipio_General Bravo": "20",
    "municipio_General Escobedo": "21",
    "municipio_General Terán": "22",
    "municipio_General Treviño": "23",
    "municipio_General Zaragoza": "24",
    "municipio_General Zuazua": "25",
    "municipio_Higueras": "28",
    "municipio_Hualahuises": "29",
    "municipio_Iturbide": "30",
    "municipio_Lampazos de Naranjo": "32",
    "municipio_Linares": "33",
    "municipio_Los Aldamas": "3",
    "municipio_Los Herreras": "27",
    "municipio_Los Ramones": "42",
    "municipio_Marín": "34",
    "municipio_Mier Y Noriega": "36",
    "municipio_Mina": "37",
    "municipio_Montemorelos": "38",
    "municipio_Monterrey": "39",
    "municipio_Parás": "40",
    "municipio_Pesquería": "41",
    "municipio_Rayones": "43",
    "municipio_Sabinas Hidalgo": "44",
    "municipio_Salinas Victoria": "45",
    "municipio_San Nicolás de los Garza": "46",
    "municipio_San Pedro Garza García": "19",
    "municipio_Santiago": "49",
    "municipio_Vallecillo": "50",
    "municipio_Villaldama": "51",
    "distrito_Cañada": "109",
    "distrito_Costa": "106",
    "distrito_Huajuapan de León": "104",
    "distrito_Istmo": "107",
    "distrito_Sierra Juárez": "108",
    "distrito_Tuxtepec": "110",
    "distrito_Valles Centrales": "105",
    "municipio_Abejones": "1",
    "municipio_Acatlán de Pérez Figueroa": "2",
    "municipio_Ánimas Trujano": "174",
    "municipio_Asunción Cacalotepec": "3",
    "municipio_Asunción Cuyotepeji": "4",
    "municipio_Asunción Ixtaltepec": "5",
    "municipio_Asunción Nochixtlán": "6",
    "municipio_Asunción Ocotlán": "7",
    "municipio_Asunción Tlacolulita": "8",
    "municipio_Ayoquezco de Aldama": "398",
    "municipio_Ayotzintepec": "9",
    "municipio_Calihualá": "11",
    "municipio_Candelaria Loxicha": "12",
    "municipio_Capulálpam de Méndez": "247",
    "municipio_Chahuites": "25",
    "municipio_Chalcatongo de Hidalgo": "26",
    "municipio_Chiquihuitlán de Benito Juárez": "27",
    "municipio_Ciénega de Zimatlán": "13",
    "municipio_Ciudad Ixtepec": "14",
    "municipio_Coatecas Altas": "15",
    "municipio_Coicoyán de Las Flores": "16",
    "municipio_Concepción Buenavista": "18",
    "municipio_Concepción Pápalo": "19",
    "municipio_Constancia del Rosario": "20",
    "municipio_Cosolapa": "21",
    "municipio_Cosoltepec": "22",
    "municipio_Cuilápam de Guerrero": "23",
    "municipio_Cuyamecalco Villa de Zaragoza": "24",
    "municipio_El Barrio de la Soledad": "10",
    "municipio_El Espinal": "30",
    "municipio_Eloxochitlán de Flores Magón": "29",
    "municipio_Fresnillo de Trujano": "32",
    "municipio_Guadalupe de Ramírez": "34",
    "municipio_Guadalupe Etla": "33",
    "municipio_Guelatao de Juárez": "35",
    "municipio_Guevea de Humboldt": "36",
    "municipio_H. V. Tezoatlán de Segura y Luna, C. de la I.de O.": "549",
    "municipio_Heroica Ciudad de Ejutla de Crespo": "28",
    "municipio_Heroica Ciudad de Huajuapan de León": "39",
    "municipio_Heroica Ciudad de Juchitán de Zaragoza": "43",
    "municipio_Heroica Ciudad de Tlaxiaco": "397",
    "municipio_Huautepec": "40",
    "municipio_Huautla de Jiménez": "41",
    "municipio_Ixpantepec Nieves": "65",
    "municipio_Ixtlán de Juárez": "42",
    "municipio_La Compañía": "17",
    "municipio_La Pe": "69",
    "municipio_La Reforma": "76",
    "municipio_La Trinidad Vista Hermosa": "556",
    "municipio_Loma Bonita": "44",
    "municipio_Magdalena Apasco": "45",
    "municipio_Magdalena Jaltepec": "46",
    "municipio_Magdalena Mixtepec": "48",
    "municipio_Magdalena Ocotlán": "49",
    "municipio_Magdalena Peñasco": "50",
    "municipio_Magdalena Teitipac": "51",
    "municipio_Magdalena Tequisistlán": "52",
    "municipio_Magdalena Tlacotepec": "53",
    "municipio_Magdalena Yodocono de Porfirio Díaz": "562",
    "municipio_Magdalena Zahuatlán": "54",
    "municipio_Mariscala de Juárez": "55",
    "municipio_Mártires de Tacubaya": "56",
    "municipio_Matías Romero Avendaño": "57",
    "municipio_Mazatlán Villa de Flores": "58",
    "municipio_Mesones Hidalgo": "37",
    "municipio_Miahuatlán de Porfirio Díaz": "59",
    "municipio_Mixistlán de La Reforma": "60",
    "municipio_Monjas": "61",
    "municipio_Natividad": "62",
    "municipio_Nazareno Etla": "63",
    "municipio_Nejapa de Madero": "64",
    "municipio_Nuevo Zoquiápam": "504",
    "municipio_Oaxaca de Juárez": "67",
    "municipio_Ocotlán de Morelos": "68",
    "municipio_Pinotepa de Don Luis": "70",
    "municipio_Pluma Hidalgo": "71",
    "municipio_Putla Villa de Guerrero": "73",
    "municipio_Reforma de Pineda": "75",
    "municipio_Reyes Etla": "77",
    "municipio_Rojas de Cuauhtémoc": "78",
    "municipio_Salina Cruz": "79",
    "municipio_San Agustín Amatengo": "80",
    "municipio_San Agustín Atenango": "81",
    "municipio_San Agustín Chayuco": "82",
    "municipio_San Agustín de Las Juntas": "83",
    "municipio_San Agustín Etla": "84",
    "municipio_San Agustín Loxicha": "85",
    "municipio_San Agustín Tlacotepec": "86",
    "municipio_San Agustín Yatareni": "87",
    "municipio_San Andrés Cabecera Nueva": "88",
    "municipio_San Andrés Dinicuiti": "89",
    "municipio_San Andrés Huaxpaltepec": "90",
    "municipio_San Andrés Huayápam": "91",
    "municipio_San Andrés Ixtlahuaca": "92",
    "municipio_San Andrés Lagunas": "93",
    "municipio_San Andrés Nuxiño": "94",
    "municipio_San Andrés Paxtlán": "95",
    "municipio_San Andrés Sinaxtla": "96",
    "municipio_San Andrés Solaga": "97",
    "municipio_San Andrés Teotilálpam": "98",
    "municipio_San Andrés Tepetlapa": "99",
    "municipio_San Andrés Yaá": "100",
    "municipio_San Andrés Zabache": "101",
    "municipio_San Andrés Zautla": "102",
    "municipio_San Antonino Castillo Velasco": "103",
    "municipio_San Antonino El Alto": "104",
    "municipio_San Antonino Monte Verde": "105",
    "municipio_San Antonio Acutla": "106",
    "municipio_San Antonio de La Cal": "107",
    "municipio_San Antonio Huitepec": "108",
    "municipio_San Antonio Nanahuatípam": "109",
    "municipio_San Antonio Sinicahua": "110",
    "municipio_San Antonio Tepetlapa": "111",
    "municipio_San Baltazar Chichicápam": "112",
    "municipio_San Baltazar Loxicha": "113",
    "municipio_San Baltazar Yatzachi El Bajo": "114",
    "municipio_San Bartolo Coyotepec": "115",
    "municipio_San Bartolo Soyaltepec": "121",
    "municipio_San Bartolo Yautepec": "122",
    "municipio_San Bartolomé Ayautla": "116",
    "municipio_San Bartolomé Loxicha": "117",
    "municipio_San Bartolomé Quialana": "118",
    "municipio_San Bartolomé Yucuañe": "119",
    "municipio_San Bartolomé Zoogocho": "120",
    "municipio_San Bernardo Mixtepec": "123",
    "municipio_San Blas Atempa": "124",
    "municipio_San Carlos Yautepec": "125",
    "municipio_San Cristóbal Amatlán": "126",
    "municipio_San Cristóbal Amoltepec": "127",
    "municipio_San Cristóbal Lachirioag": "128",
    "municipio_San Cristóbal Suchixtlahuaca": "129",
    "municipio_San Dionisio del Mar": "130",
    "municipio_San Dionisio Ocotepec": "131",
    "municipio_San Dionisio Ocotlán": "132",
    "municipio_San Esteban Atatlahuca": "133",
    "municipio_San Felipe Jalapa de Díaz": "134",
    "municipio_San Felipe Tejalápam": "135",
    "municipio_San Felipe Usila": "136",
    "municipio_San Francisco Cahuacuá": "137",
    "municipio_San Francisco Cajonos": "138",
    "municipio_San Francisco Chapulapa": "139",
    "municipio_San Francisco Chindúa": "140",
    "municipio_San Francisco del Mar": "141",
    "municipio_San Francisco Huehuetlán": "142",
    "municipio_San Francisco Ixhuatán": "143",
    "municipio_San Francisco Jaltepetongo": "144",
    "municipio_San Francisco Lachigoló": "145",
    "municipio_San Francisco Logueche": "146",
    "municipio_San Francisco Nuxaño": "147",
    "municipio_San Francisco Ozolotepec": "148",
    "municipio_San Francisco Sola": "149",
    "municipio_San Francisco Telixtlahuaca": "150",
    "municipio_San Francisco Teopan": "151",
    "municipio_San Francisco Tlapancingo": "152",
    "municipio_San Gabriel Mixtepec": "153",
    "municipio_San Ildefonso Amatlán": "154",
    "municipio_San Ildefonso Sola": "155",
    "municipio_San Ildefonso Villa Alta": "156",
    "municipio_San Jacinto Amilpas": "157",
    "municipio_San Jacinto Tlacotepec": "158",
    "municipio_San Jerónimo Coatlán": "159",
    "municipio_San Jerónimo Silacayoapilla": "160",
    "municipio_San Jerónimo Sosola": "161",
    "municipio_San Jerónimo Taviche": "162",
    "municipio_San Jerónimo Tecóatl": "163",
    "municipio_San Jerónimo Tlacochahuaya": "550",
    "municipio_San Jorge Nuchita": "164",
    "municipio_San José Ayuquila": "165",
    "municipio_San José Chiltepec": "166",
    "municipio_San José del Peñasco": "167",
    "municipio_San José del Progreso": "72",
    "municipio_San José Estancia Grande": "168",
    "municipio_San José Independencia": "169",
    "municipio_San José Lachiguiri": "170",
    "municipio_San José Tenango": "171",
    "municipio_San Juan Achiutla": "172",
    "municipio_San Juan Atepec": "173",
    "municipio_San Juan Bautista Atatlahuca": "175",
    "municipio_San Juan Bautista Coixtlahuaca": "176",
    "municipio_San Juan Bautista Cuicatlán": "177",
    "municipio_San Juan Bautista Guelache": "178",
    "municipio_San Juan Bautista Jayacatlán": "179",
    "municipio_San Juan Bautista Lo de Soto": "180",
    "municipio_San Juan Bautista Suchitepec": "181",
    "municipio_San Juan Bautista Tlachichilco": "183",
    "municipio_San Juan Bautista Tlacoatzintepec": "182",
    "municipio_San Juan Bautista Tuxtepec": "184",
    "municipio_San Juan Bautista Valle Nacional": "559",
    "municipio_San Juan Cacahuatepec": "185",
    "municipio_San Juan Chicomezúchil": "191",
    "municipio_San Juan Chilateca": "192",
    "municipio_San Juan Cieneguilla": "186",
    "municipio_San Juan Coatzóspam": "187",
    "municipio_San Juan Colorado": "188",
    "municipio_San Juan Comaltepec": "189",
    "municipio_San Juan Cotzocón": "190",
    "municipio_San Juan de Los Cués": "206",
    "municipio_San Juan del Estado": "193",
    "municipio_San Juan Diuxi": "195",
    "municipio_San Juan Evangelista Analco": "196",
    "municipio_San Juan Guelavía": "197",
    "municipio_San Juan Guichicovi": "198",
    "municipio_San Juan Ihualtepec": "199",
    "municipio_San Juan Juquila Mixes": "200",
    "municipio_San Juan Juquila Vijanos": "201",
    "municipio_San Juan Lachao": "202",
    "municipio_San Juan Lachigalla": "203",
    "municipio_San Juan Lajarcia": "204",
    "municipio_San Juan Lalana": "205",
    "municipio_San Juan Mazatlán": "207",
    "municipio_San Juan Mixtepec": "208",
    "municipio_San Juan Ñumí": "210",
    "municipio_San Juan Ozolotepec": "211",
    "municipio_San Juan Petlapa": "212",
    "municipio_San Juan Quiahije": "213",
    "municipio_San Juan Quiotepec": "214",
    "municipio_San Juan Sayultepec": "215",
    "municipio_San Juan Tabaá": "216",
    "municipio_San Juan Tamazola": "217",
    "municipio_San Juan Teita": "218",
    "municipio_San Juan Teitipac": "219",
    "municipio_San Juan Tepeuxila": "220",
    "municipio_San Juan Teposcolula": "221",
    "municipio_San Juan Yaeé": "222",
    "municipio_San Juan Yatzona": "223",
    "municipio_San Juan Yucuita": "224",
    "municipio_San Lorenzo": "225",
    "municipio_San Lorenzo Albarradas": "226",
    "municipio_San Lorenzo Cacaotepec": "227",
    "municipio_San Lorenzo Cuaunecuiltitla": "228",
    "municipio_San Lorenzo Texmelúcan": "229",
    "municipio_San Lorenzo Victoria": "230",
    "municipio_San Lucas Camotlán": "231",
    "municipio_San Lucas Ojitlán": "232",
    "municipio_San Lucas Quiaviní": "233",
    "municipio_San Lucas Zoquiápam": "234",
    "municipio_San Luis Amatlán": "235",
    "municipio_San Marcial Ozolotepec": "236",
    "municipio_San Marcos Arteaga": "237",
    "municipio_San Martín de Los Cansecos": "238",
    "municipio_San Martín Huamelúlpam": "239",
    "municipio_San Martín Itunyoso": "240",
    "municipio_San Martín Lachilá": "241",
    "municipio_San Martín Peras": "242",
    "municipio_San Martín Tilcajete": "243",
    "municipio_San Martín Toxpalan": "244",
    "municipio_San Martín Zacatepec": "245",
    "municipio_San Mateo Cajonos": "246",
    "municipio_San Mateo del Mar": "248",
    "municipio_San Mateo Etlatongo": "250",
    "municipio_San Mateo Nejápam": "251",
    "municipio_San Mateo Peñasco": "252",
    "municipio_San Mateo Piñas": "253",
    "municipio_San Mateo Río Hondo": "254",
    "municipio_San Mateo Sindihui": "255",
    "municipio_San Mateo Tlapiltepec": "256",
    "municipio_San Mateo Yoloxochitlán": "249",
    "municipio_San Mateo Yucutindoo": "566",
    "municipio_San Melchor Betaza": "257",
    "municipio_San Miguel Achiutla": "258",
    "municipio_San Miguel Ahuehuetitlán": "259",
    "municipio_San Miguel Aloápam": "260",
    "municipio_San Miguel Amatitlán": "261",
    "municipio_San Miguel Amatlán": "262",
    "municipio_San Miguel Chicahua": "264",
    "municipio_San Miguel Chimalapa": "265",
    "municipio_San Miguel Coatlán": "263",
    "municipio_San Miguel del Puerto": "266",
    "municipio_San Miguel del Río": "267",
    "municipio_San Miguel Ejutla": "268",
    "municipio_San Miguel El Grande": "269",
    "municipio_San Miguel Huautla": "270",
    "municipio_San Miguel Mixtepec": "271",
    "municipio_San Miguel Panixtlahuaca": "272",
    "municipio_San Miguel Peras": "273",
    "municipio_San Miguel Piedras": "274",
    "municipio_San Miguel Quetzaltepec": "275",
    "municipio_San Miguel Santa Flor": "276",
    "municipio_San Miguel Soyaltepec": "278",
    "municipio_San Miguel Suchixtepec": "279",
    "municipio_San Miguel Tecomatlán": "281",
    "municipio_San Miguel Tenango": "282",
    "municipio_San Miguel Tequixtepec": "283",
    "municipio_San Miguel Tilquiápam": "284",
    "municipio_San Miguel Tlacamama": "285",
    "municipio_San Miguel Tlacotepec": "286",
    "municipio_San Miguel Tulancingo": "287",
    "municipio_San Miguel Yotao": "288",
    "municipio_San Nicolás": "36",
    "municipio_San Nicolás Hidalgo": "290",
    "municipio_San Pablo Coatlán": "291",
    "municipio_San Pablo Cuatro Venados": "292",
    "municipio_San Pablo Etla": "293",
    "municipio_San Pablo Huitzo": "294",
    "municipio_San Pablo Huixtepec": "295",
    "municipio_San Pablo Macuiltianguis": "296",
    "municipio_San Pablo Tijaltepec": "297",
    "municipio_San Pablo Villa de Mitla": "298",
    "municipio_San Pablo Yaganiza": "299",
    "municipio_San Pedro Amuzgos": "300",
    "municipio_San Pedro Apóstol": "301",
    "municipio_San Pedro Atoyac": "302",
    "municipio_San Pedro Cajonos": "303",
    "municipio_San Pedro Comitancillo": "305",
    "municipio_San Pedro Coxcaltepec Cántaros": "304",
    "municipio_San Pedro El Alto": "306",
    "municipio_San Pedro Huamelula": "307",
    "municipio_San Pedro Huilotepec": "308",
    "municipio_San Pedro Ixcatlán": "309",
    "municipio_San Pedro Ixtlahuaca": "310",
    "municipio_San Pedro Jaltepetongo": "311",
    "municipio_San Pedro Jicayán": "312",
    "municipio_San Pedro Jocotipac": "313",
    "municipio_San Pedro Juchatengo": "314",
    "municipio_San Pedro Mártir": "315",
    "municipio_San Pedro Mártir Quiechapa": "316",
    "municipio_San Pedro Mártir Yucuxaco": "317",
    "municipio_San Pedro Mixtepec": "318",
    "municipio_San Pedro Molinos": "320",
    "municipio_San Pedro Nopala": "321",
    "municipio_San Pedro Ocopetatillo": "322",
    "municipio_San Pedro Ocotepec": "323",
    "municipio_San Pedro Pochutla": "324",
    "municipio_San Pedro Quiatoni": "325",
    "municipio_San Pedro Sochiápam": "326",
    "municipio_San Pedro Tapanatepec": "327",
    "municipio_San Pedro Taviche": "328",
    "municipio_San Pedro Teozacoalco": "329",
    "municipio_San Pedro Teutila": "330",
    "municipio_San Pedro Tidaá": "331",
    "municipio_San Pedro Topiltepec": "332",
    "municipio_San Pedro Totolápam": "333",
    "municipio_San Pedro Y San Pablo Ayutla": "337",
    "municipio_San Pedro Y San Pablo Teposcolula": "339",
    "municipio_San Pedro Y San Pablo Tequixtepec": "340",
    "municipio_San Pedro Yaneri": "335",
    "municipio_San Pedro Yólox": "336",
    "municipio_San Pedro Yucunama": "341",
    "municipio_San Raymundo Jalpan": "342",
    "municipio_San Sebastián Abasolo": "343",
    "municipio_San Sebastián Coatlán": "344",
    "municipio_San Sebastián Ixcapa": "345",
    "municipio_San Sebastián Nicananduta": "346",
    "municipio_San Sebastián Río Hondo": "347",
    "municipio_San Sebastián Tecomaxtlahuaca": "348",
    "municipio_San Sebastián Teitipac": "349",
    "municipio_San Sebastián Tutla": "350",
    "municipio_San Simón Almolongas": "351",
    "municipio_San Simón Zahuatlán": "352",
    "municipio_San Vicente Coatlán": "534",
    "municipio_San Vicente Lachixío": "535",
    "municipio_San Vicente Nuñú": "536",
    "municipio_Santa Ana": "58",
    "municipio_Santa Ana Ateixtlahuaca": "354",
    "municipio_Santa Ana Cuauhtémoc": "355",
    "municipio_Santa Ana del Valle": "356",
    "municipio_Santa Ana Tavela": "357",
    "municipio_Santa Ana Tlapacoyan": "358",
    "municipio_Santa Ana Yareni": "359",
    "municipio_Santa Ana Zegache": "360",
    "municipio_Santa Catalina Quierí": "361",
    "municipio_Santa Catarina Cuixtla": "362",
    "municipio_Santa Catarina Ixtepeji": "363",
    "municipio_Santa Catarina Juquila": "364",
    "municipio_Santa Catarina Lachatao": "365",
    "municipio_Santa Catarina Loxicha": "366",
    "municipio_Santa Catarina Mechoacán": "367",
    "municipio_Santa Catarina Minas": "368",
    "municipio_Santa Catarina Quiané": "369",
    "municipio_Santa Catarina Quioquitani": "74",
    "municipio_Santa Catarina Tayata": "370",
    "municipio_Santa Catarina Ticuá": "371",
    "municipio_Santa Catarina Yosonotú": "372",
    "municipio_Santa Catarina Zapoquila": "373",
    "municipio_Santa Cruz Acatepec": "374",
    "municipio_Santa Cruz Amilpas": "375",
    "municipio_Santa Cruz de Bravo": "376",
    "municipio_Santa Cruz Itundujia": "377",
    "municipio_Santa Cruz Mixtepec": "378",
    "municipio_Santa Cruz Nundaco": "379",
    "municipio_Santa Cruz Papalutla": "380",
    "municipio_Santa Cruz Tacache de Mina": "381",
    "municipio_Santa Cruz Tacahua": "382",
    "municipio_Santa Cruz Tayata": "383",
    "municipio_Santa Cruz Xitla": "384",
    "municipio_Santa Cruz Xoxocotlán": "385",
    "municipio_Santa Cruz Zenzontepec": "386",
    "municipio_Santa Gertrudis": "387",
    "municipio_Santa Inés de Zaragoza": "569",
    "municipio_Santa Inés del Monte": "388",
    "municipio_Santa Inés Yatzeche": "389",
    "municipio_Santa Lucía del Camino": "390",
    "municipio_Santa Lucía Miahuatlán": "391",
    "municipio_Santa Lucía Monteverde": "392",
    "municipio_Santa Lucía Ocotlán": "393",
    "municipio_Santa Magdalena Jicotlán": "47",
    "municipio_Santa María Alotepec": "394",
    "municipio_Santa María Apazco": "395",
    "municipio_Santa María Atzompa": "399",
    "municipio_Santa María Camotlán": "400",
    "municipio_Santa María Chachoápam": "404",
    "municipio_Santa María Chilchotla": "406",
    "municipio_Santa María Chimalapa": "407",
    "municipio_Santa María Colotepec": "401",
    "municipio_Santa María Cortijo": "402",
    "municipio_Santa María Coyotepec": "403",
    "municipio_Santa María del Rosario": "408",
    "municipio_Santa María del Tule": "409",
    "municipio_Santa María Ecatepec": "410",
    "municipio_Santa María Guelacé": "411",
    "municipio_Santa María Guienagati": "412",
    "municipio_Santa María Huatulco": "413",
    "municipio_Santa María Huazolotitlán": "414",
    "municipio_Santa María Ipalapa": "415",
    "municipio_Santa María Ixcatlán": "416",
    "municipio_Santa María Jacatepec": "417",
    "municipio_Santa María Jalapa del Marqués": "418",
    "municipio_Santa María Jaltianguis": "419",
    "municipio_Santa María La Asunción": "396",
    "municipio_Santa María Lachixío": "420",
    "municipio_Santa María Mixtequilla": "421",
    "municipio_Santa María Nativitas": "422",
    "municipio_Santa María Nduayaco": "423",
    "municipio_Santa María Ozolotepec": "424",
    "municipio_Santa María Pápalo": "425",
    "municipio_Santa María Peñoles": "426",
    "municipio_Santa María Petapa": "427",
    "municipio_Santa María Quiegolani": "428",
    "municipio_Santa María Sola": "429",
    "municipio_Santa María Tataltepec": "430",
    "municipio_Santa María Tecomavaca": "431",
    "municipio_Santa María Temaxcalapa": "432",
    "municipio_Santa María Temaxcaltepec": "433",
    "municipio_Santa María Teopoxco": "434",
    "municipio_Santa María Tepantlali": "435",
    "municipio_Santa María Texcatitlán": "436",
    "municipio_Santa María Tlahuitoltepec": "437",
    "municipio_Santa María Tlalixtac": "438",
    "municipio_Santa María Tonameca": "439",
    "municipio_Santa María Totolapilla": "440",
    "municipio_Santa María Xadani": "441",
    "municipio_Santa María Yalina": "442",
    "municipio_Santa María Yavesía": "443",
    "municipio_Santa María Yolotepec": "444",
    "municipio_Santa María Yosoyúa": "445",
    "municipio_Santa María Yucuhiti": "446",
    "municipio_Santa María Zacatepec": "447",
    "municipio_Santa María Zaniza": "448",
    "municipio_Santa María Zoquitlán": "449",
    "municipio_Santiago Amoltepec": "450",
    "municipio_Santiago Apoala": "451",
    "municipio_Santiago Apóstol": "452",
    "municipio_Santiago Astata": "453",
    "municipio_Santiago Atitlán": "454",
    "municipio_Santiago Ayuquililla": "455",
    "municipio_Santiago Cacaloxtepec": "456",
    "municipio_Santiago Camotlán": "457",
    "municipio_Santiago Chazumba": "459",
    "municipio_Santiago Choápam": "460",
    "municipio_Santiago Comaltepec": "458",
    "municipio_Santiago del Río": "461",
    "municipio_Santiago Huajolotitlán": "462",
    "municipio_Santiago Huauclilla": "463",
    "municipio_Santiago Ihuitlán Plumas": "464",
    "municipio_Santiago Ixcuintepec": "465",
    "municipio_Santiago Ixtayutla": "466",
    "municipio_Santiago Jamiltepec": "467",
    "municipio_Santiago Jocotepec": "468",
    "municipio_Santiago Juxtlahuaca": "469",
    "municipio_Santiago Lachiguiri": "470",
    "municipio_Santiago Lalopa": "471",
    "municipio_Santiago Laollaga": "472",
    "municipio_Santiago Laxopa": "473",
    "municipio_Santiago Llano Grande": "474",
    "municipio_Santiago Matatlán": "475",
    "municipio_Santiago Miltepec": "476",
    "municipio_Santiago Minas": "477",
    "municipio_Santiago Nacaltepec": "478",
    "municipio_Santiago Nejapilla": "479",
    "municipio_Santiago Niltepec": "66",
    "municipio_Santiago Nundiche": "480",
    "municipio_Santiago Nuyoó": "481",
    "municipio_Santiago Pinotepa Nacional": "482",
    "municipio_Santiago Suchilquitongo": "483",
    "municipio_Santiago Tamazola": "484",
    "municipio_Santiago Tapextla": "485",
    "municipio_Santiago Tenango": "487",
    "municipio_Santiago Tepetlapa": "488",
    "municipio_Santiago Tetepec": "489",
    "municipio_Santiago Texcalcingo": "490",
    "municipio_Santiago Textitlán": "491",
    "municipio_Santiago Tilantongo": "492",
    "municipio_Santiago Tillo": "493",
    "municipio_Santiago Tlazoyaltepec": "494",
    "municipio_Santiago Xanica": "495",
    "municipio_Santiago Xiacuí": "496",
    "municipio_Santiago Yaitepec": "497",
    "municipio_Santiago Yaveo": "498",
    "municipio_Santiago Yolomécatl": "499",
    "municipio_Santiago Yosondúa": "500",
    "municipio_Santiago Yucuyachi": "501",
    "municipio_Santiago Zacatepec": "502",
    "municipio_Santiago Zoochila": "503",
    "municipio_Santo Domingo Albarradas": "506",
    "municipio_Santo Domingo Armenta": "507",
    "municipio_Santo Domingo Chihuitán": "508",
    "municipio_Santo Domingo de Morelos": "509",
    "municipio_Santo Domingo Ingenio": "505",
    "municipio_Santo Domingo Ixcatlán": "510",
    "municipio_Santo Domingo Nuxaá": "511",
    "municipio_Santo Domingo Ozolotepec": "512",
    "municipio_Santo Domingo Petapa": "513",
    "municipio_Santo Domingo Roayaga": "514",
    "municipio_Santo Domingo Tehuantepec": "515",
    "municipio_Santo Domingo Teojomulco": "516",
    "municipio_Santo Domingo Tepuxtepec": "517",
    "municipio_Santo Domingo Tlatayápam": "518",
    "municipio_Santo Domingo Tomaltepec": "519",
    "municipio_Santo Domingo Tonalá": "520",
    "municipio_Santo Domingo Tonaltepec": "521",
    "municipio_Santo Domingo Xagacía": "522",
    "municipio_Santo Domingo Yanhuitlán": "523",
    "municipio_Santo Domingo Yodohino": "524",
    "municipio_Santo Domingo Zanatepec": "525",
    "municipio_Santo Tomás Jalieza": "530",
    "municipio_Santo Tomás Mazaltepec": "531",
    "municipio_Santo Tomás Ocotepec": "532",
    "municipio_Santo Tomás Tamazulapan": "533",
    "municipio_Santos Reyes Nopala": "526",
    "municipio_Santos Reyes Pápalo": "527",
    "municipio_Santos Reyes Tepejillo": "528",
    "municipio_Santos Reyes Yucuná": "529",
    "municipio_Silacayoápam": "537",
    "municipio_Sitio de Xitlapehua": "538",
    "municipio_Soledad Etla": "539",
    "municipio_Tamazulápam del Espíritu Santo": "31",
    "municipio_Tanetze de Zaragoza": "541",
    "municipio_Taniche": "542",
    "municipio_Tataltepec de Valdés": "543",
    "municipio_Teococuilco de Marcos Pérez": "544",
    "municipio_Teotitlán de Flores Magón": "545",
    "municipio_Teotitlán del Valle": "546",
    "municipio_Teotongo": "547",
    "municipio_Tepelmeme Villa de Morelos": "548",
    "municipio_Tlacolula de Matamoros": "551",
    "municipio_Tlacotepec Plumas": "552",
    "municipio_Tlalixtac de Cabrera": "553",
    "municipio_Totontepec Villa de Morelos": "554",
    "municipio_Trinidad Zaachila": "555",
    "municipio_Unión Hidalgo": "557",
    "municipio_Valerio Trujano": "558",
    "municipio_Villa de Chilapa de Díaz": "405",
    "municipio_Villa de Etla": "338",
    "municipio_Villa de Tamazulápam del Progreso": "540",
    "municipio_Villa de Tututepec de Melchor Ocampo": "334",
    "municipio_Villa de Zaachila": "565",
    "municipio_Villa Díaz Ordaz": "560",
    "municipio_Villa Sola de Vega": "277",
    "municipio_Villa Talea de Castro": "280",
    "municipio_Villa Tejúpam de La Unión": "486",
    "municipio_Yaxe": "561",
    "municipio_Yogana": "563",
    "municipio_Yutanduchi de Guerrero": "564",
    "municipio_Zapotitlán Lagunas": "567",
    "municipio_Zapotitlán Palmas": "568",
    "municipio_Zimatlán de Álvarez": "570",
    "distrito_Cholula": "115",
    "distrito_Huauchinango": "111",
    "distrito_Izúcar de Matamoros": "116",
    "distrito_Libres": "114",
    "distrito_Tecamachalco": "117",
    "distrito_Tehuacán": "118",
    "distrito_Teziutlán": "113",
    "distrito_Zacatlán": "112",
    "municipio_Acajete": "1",
    "municipio_Acateno": "2",
    "municipio_Acatzingo": "4",
    "municipio_Acteopan": "5",
    "municipio_Ahuatlán": "7",
    "municipio_Ahuazotepec": "8",
    "municipio_Ahuehuetitla": "9",
    "municipio_Ajalpan": "10",
    "municipio_Albino Zertuche": "11",
    "municipio_Aljojuca": "12",
    "municipio_Altepexi": "13",
    "municipio_Amixtlán": "14",
    "municipio_Amozoc": "15",
    "municipio_Aquixtla": "16",
    "municipio_Atempan": "17",
    "municipio_Atexcal": "18",
    "municipio_Atlequizayan": "80",
    "municipio_Atlixco": "19",
    "municipio_Atoyatempan": "20",
    "municipio_Atzala": "21",
    "municipio_Atzitzihuacán": "22",
    "municipio_Atzitzintla": "23",
    "municipio_Axutla": "24",
    "municipio_Ayotoxco de Guerrero": "25",
    "municipio_Calpan": "26",
    "municipio_Caltepec": "27",
    "municipio_Camocuautla": "28",
    "municipio_Cañada Morelos": "99",
    "municipio_Caxhuacan": "29",
    "municipio_Chalchicomula de Sesma": "45",
    "municipio_Chapulco": "46",
    "municipio_Chiautzingo": "48",
    "municipio_Chichiquila": "50",
    "municipio_Chiconcuautla": "49",
    "municipio_Chietla": "51",
    "municipio_Chigmecatitlán": "52",
    "municipio_Chignahuapan": "53",
    "municipio_Chignautla": "54",
    "municipio_Chila": "55",
    "municipio_Chila de La Sal": "56",
    "municipio_Chilchotla": "58",
    "municipio_Chinantla": "59",
    "municipio_Coatepec": "38",
    "municipio_Coatzingo": "31",
    "municipio_Cohetzala": "32",
    "municipio_Cohuecan": "33",
    "municipio_Coronango": "34",
    "municipio_Coxcatlán": "14",
    "municipio_Coyomeapan": "36",
    "municipio_Cuapiaxtla de Madero": "38",
    "municipio_Cuautempan": "39",
    "municipio_Cuautinchán": "40",
    "municipio_Cuautlancingo": "41",
    "municipio_Cuayuca de Andrade": "42",
    "municipio_Cuetzalan del Progreso": "43",
    "municipio_Cuyoaco": "44",
    "municipio_Domingo Arenas": "60",
    "municipio_Epatlán": "62",
    "municipio_Esperanza": "63",
    "municipio_Francisco Z. Mena": "64",
    "municipio_General Felipe Ángeles": "65",
    "municipio_Hermenegildo Galeana": "68",
    "municipio_Honey": "57",
    "municipio_Huaquechula": "69",
    "municipio_Huatlatlauca": "70",
    "municipio_Huauchinango": "71",
    "municipio_Huehuetlán El Chico": "73",
    "municipio_Huehuetlán El Grande": "150",
    "municipio_Huejotzingo": "74",
    "municipio_Hueytamalco": "76",
    "municipio_Hueytlalpan": "77",
    "municipio_Huitzilan de Serdán": "78",
    "municipio_Huitziltepec": "79",
    "municipio_Ixcamilpa de Guerrero": "81",
    "municipio_Ixcaquixtla": "82",
    "municipio_Ixtacamaxtitlán": "83",
    "municipio_Ixtepec": "84",
    "municipio_Izúcar de Matamoros": "85",
    "municipio_Jalpan": "86",
    "municipio_Jolalpan": "87",
    "municipio_Jonotla": "88",
    "municipio_Jopala": "89",
    "municipio_Juan C. Bonilla": "90",
    "municipio_Juan Galindo": "91",
    "municipio_Juan N. Méndez": "92",
    "municipio_La Magdalena Tlatlauquitepec": "95",
    "municipio_Lafragua": "93",
    "municipio_Libres": "94",
    "municipio_Los Reyes de Juárez": "118",
    "municipio_Mazapiltepec de Juárez": "96",
    "municipio_Mixtla": "97",
    "municipio_Molcaxac": "98",
    "municipio_Naupan": "100",
    "municipio_Nauzontla": "101",
    "municipio_Nealtican": "102",
    "municipio_Nicolás Bravo": "103",
    "municipio_Nopalucan": "104",
    "municipio_Ocoyucan": "106",
    "municipio_Olintla": "107",
    "municipio_Oriental": "108",
    "municipio_Pahuatlán": "109",
    "municipio_Palmar de Bravo": "110",
    "municipio_Petlalcingo": "112",
    "municipio_Piaxtla": "113",
    "municipio_Puebla": "114",
    "municipio_Quecholac": "115",
    "municipio_Quimixtlán": "116",
    "municipio_Rafael Lara Grajales": "117",
    "municipio_San Andrés Cholula": "119",
    "municipio_San Antonio Cañada": "120",
    "municipio_San Diego La Mesa Tochimiltzingo": "121",
    "municipio_San Felipe Teotlalcingo": "122",
    "municipio_San Felipe Tepatlán": "123",
    "municipio_San Gabriel Chilac": "124",
    "municipio_San Gregorio Atzompa": "125",
    "municipio_San Jerónimo Tecuanipan": "126",
    "municipio_San Jerónimo Xayacatlán": "127",
    "municipio_San José Chiapa": "128",
    "municipio_San José Miahuatlán": "129",
    "municipio_San Juan Atenco": "130",
    "municipio_San Juan Atzompa": "131",
    "municipio_San Martín Texmelucan": "132",
    "municipio_San Martín Totoltepec": "133",
    "municipio_San Matías Tlalancaleca": "134",
    "municipio_San Miguel Ixitlán": "135",
    "municipio_San Miguel Xoxtla": "136",
    "municipio_San Nicolás Buenos Aires": "137",
    "municipio_San Nicolás de Los Ranchos": "138",
    "municipio_San Pablo Anicano": "139",
    "municipio_San Pedro Cholula": "140",
    "municipio_San Pedro Yeloixtlahuaca": "141",
    "municipio_San Salvador El Seco": "142",
    "municipio_San Salvador El Verde": "143",
    "municipio_San Salvador Huixcolotla": "144",
    "municipio_San Sebastián Tlacotepec": "145",
    "municipio_Santa Catarina Tlaltempan": "146",
    "municipio_Santa Inés Ahuatempan": "147",
    "municipio_Santa Isabel Cholula": "148",
    "municipio_Santiago Miahuatlán": "149",
    "municipio_Santo Tomás Hueyotlipan": "151",
    "municipio_Soltepec": "152",
    "municipio_Tecali de Herrera": "153",
    "municipio_Tecamachalco": "154",
    "municipio_Tecomatlán": "155",
    "municipio_Tehuacán": "156",
    "municipio_Tehuitzingo": "157",
    "municipio_Tenampulco": "158",
    "municipio_Teopantlán": "159",
    "municipio_Teotlalco": "160",
    "municipio_Tepanco de López": "161",
    "municipio_Tepango de Rodríguez": "162",
    "municipio_Tepatlaxco de Hidalgo": "163",
    "municipio_Tepeaca": "164",
    "municipio_Tepemaxalco": "165",
    "municipio_Tepeojuma": "166",
    "municipio_Tepetzintla": "167",
    "municipio_Tepexco": "168",
    "municipio_Tepexi de Rodríguez": "169",
    "municipio_Tepeyahualco": "170",
    "municipio_Tepeyahualco de Cuauhtémoc": "171",
    "municipio_Tetela de Ocampo": "172",
    "municipio_Teteles de Avila Castillo": "173",
    "municipio_Teziutlán": "174",
    "municipio_Tianguismanalco": "175",
    "municipio_Tilapa": "176",
    "municipio_Tlachichuca": "179",
    "municipio_Tlacotepec de Benito Juárez": "177",
    "municipio_Tlacuilotepec": "178",
    "municipio_Tlahuapan": "180",
    "municipio_Tlaltenango": "181",
    "municipio_Tlanepantla": "182",
    "municipio_Tlaola": "183",
    "municipio_Tlapacoya": "184",
    "municipio_Tlapanalá": "185",
    "municipio_Tlatlauquitepec": "186",
    "municipio_Tlaxco": "34",
    "municipio_Tochimilco": "188",
    "municipio_Tochtepec": "189",
    "municipio_Totoltepec de Guerrero": "190",
    "municipio_Tulcingo": "191",
    "municipio_Tuzamapan de Galeana": "192",
    "municipio_Tzicatlacoyan": "193",
    "municipio_Xayacatlán de Bravo": "196",
    "municipio_Xicotepec": "197",
    "municipio_Xicotlán": "198",
    "municipio_Xiutetelco": "199",
    "municipio_Xochiapulco": "200",
    "municipio_Xochiltepec": "201",
    "municipio_Xochitlán de Vicente Suárez": "202",
    "municipio_Xochitlán Todos Santos": "203",
    "municipio_Yaonáhuac": "204",
    "municipio_Yehualtepec": "205",
    "municipio_Zacapala": "206",
    "municipio_Zacapoaxtla": "207",
    "municipio_Zacatlán": "208",
    "municipio_Zapotitlán": "209",
    "municipio_Zapotitlán de Méndez": "210",
    "municipio_Zautla": "212",
    "municipio_Zihuateutla": "213",
    "municipio_Zinacatepec": "214",
    "municipio_Zongozotla": "215",
    "municipio_Zoquiapan": "216",
    "municipio_Zoquitlán": "217",
    "distrito_Cadereyta": "120",
    "distrito_Jalpan": "119",
    "distrito_Querétaro": "122",
    "distrito_San Juan del Río": "121",
    "municipio_Amealco de Bonfil": "1",
    "municipio_Arroyo Seco": "3",
    "municipio_Cadereyta de Montes": "4",
    "municipio_Colón": "5",
    "municipio_Corregidora": "6",
    "municipio_El Marqués": "11",
    "municipio_Ezequiel Montes": "7",
    "municipio_Huimilpan": "8",
    "municipio_Jalpan de Serra": "9",
    "municipio_Landa de Matamoros": "10",
    "municipio_Pedro Escobedo": "12",
    "municipio_Peñamiller": "13",
    "municipio_Pinal de Amoles": "2",
    "municipio_Querétaro": "14",
    "municipio_San Joaquín": "15",
    "municipio_Tequisquiapan": "17",
    "distrito_Cancún": "125",
    "distrito_Chetumal": "123",
    "distrito_Felipe Carrillo Puerto": "124",
    "municipio_Bacalar": "10",
    "municipio_Cozumel": "1",
    "municipio_Felipe Carrillo Puerto": "2",
    "municipio_Isla Mujeres": "3",
    "municipio_José María Morelos": "6",
    "municipio_Othón P. Blanco": "4",
    "municipio_Puerto Morelos": "11",
    "municipio_Solidaridad": "8",
    "municipio_Tulum": "9",
    "distrito_Ciudad Fernández": "130",
    "distrito_Ciudad Valles": "131",
    "distrito_Ebano": "132",
    "distrito_Matehuala": "128",
    "distrito_Rioverde": "129",
    "distrito_Salinas": "127",
    "distrito_San Luis Potosí": "126",
    "municipio_Ahualulco": "1",
    "municipio_Alaquines": "2",
    "municipio_Aquismón": "3",
    "municipio_Armadillo de Los Infante": "4",
    "municipio_Axtla de Terrazas": "53",
    "municipio_Cárdenas": "2",
    "municipio_Catorce": "6",
    "municipio_Cedral": "7",
    "municipio_Cerritos": "8",
    "municipio_Cerro de San Pedro": "9",
    "municipio_Charcas": "15",
    "municipio_Ciudad del Maíz": "10",
    "municipio_Ciudad Fernández": "11",
    "municipio_Ciudad Valles": "13",
    "municipio_Ebano": "16",
    "municipio_El Naranjo": "58",
    "municipio_Guadalcázar": "17",
    "municipio_Huehuetlán": "18",
    "municipio_Matehuala": "20",
    "municipio_Matlapa": "57",
    "municipio_Mexquitic de Carmona": "21",
    "municipio_Moctezuma": "38",
    "municipio_Rioverde": "24",
    "municipio_Salinas": "25",
    "municipio_San Antonio": "26",
    "municipio_San Ciro de Acosta": "27",
    "municipio_San Luis Potosí": "28",
    "municipio_San Martín Chalchicuautla": "29",
    "municipio_San Nicolás Tolentino": "30",
    "municipio_San Vicente Tancuayalab": "34",
    "municipio_Santa María del Río": "32",
    "municipio_Santo Domingo": "33",
    "municipio_Soledad de Graciano Sánchez": "35",
    "municipio_Tamasopo": "36",
    "municipio_Tamazunchale": "37",
    "municipio_Tampacán": "38",
    "municipio_Tampamolón Corona": "39",
    "municipio_Tamuín": "40",
    "municipio_Tancanhuitz": "12",
    "municipio_Tanlajás": "41",
    "municipio_Tanquián de Escobedo": "42",
    "municipio_Tierra Nueva": "43",
    "municipio_Vanegas": "44",
    "municipio_Venado": "45",
    "municipio_Villa de Arista": "56",
    "municipio_Villa de Arriaga": "46",
    "municipio_Villa de Guadalupe": "47",
    "municipio_Villa de La Paz": "48",
    "municipio_Villa de Ramos": "49",
    "municipio_Villa de Reyes": "50",
    "municipio_Villa Juárez": "52",
    "municipio_Xilitla": "54",
    "distrito_Culiacán": "136",
    "distrito_Guamúchil": "135",
    "distrito_Guasave": "134",
    "distrito_La Cruz": "137",
    "distrito_Los Mochis": "133",
    "distrito_Mazatlán": "138",
    "municipio_Ahome": "1",
    "municipio_Angostura": "2",
    "municipio_Badiraguato": "3",
    "municipio_Choix": "7",
    "municipio_Concordia": "4",
    "municipio_Cosalá": "5",
    "municipio_Culiacán": "6",
    "municipio_El Fuerte": "10",
    "municipio_Elota": "8",
    "municipio_Escuinapa": "9",
    "municipio_Guasave": "11",
    "municipio_Mazatlán": "12",
    "municipio_Mocorito": "13",
    "municipio_Navolato": "18",
    "municipio_Salvador Alvarado": "15",
    "municipio_San Ignacio": "16",
    "municipio_Sinaloa": "17",
    "distrito_Agua Prieta": "141",
    "distrito_Caborca": "139",
    "distrito_Cajeme": "148",
    "distrito_Guaymas": "147",
    "distrito_Hermosillo": "144",
    "distrito_Magdalena": "140",
    "distrito_Mazatán": "145",
    "distrito_Moctezuma": "143",
    "distrito_Navojoa": "149",
    "distrito_Sahuaripa": "146",
    "distrito_San Luis Río Colorado": "193",
    "distrito_Ures": "142",
    "municipio_Aconchi": "1",
    "municipio_Agua Prieta": "2",
    "municipio_Alamos": "3",
    "municipio_Altar": "4",
    "municipio_Arivechi": "5",
    "municipio_Arizpe": "6",
    "municipio_Atil": "7",
    "municipio_Bacadéhuachi": "8",
    "municipio_Bacanora": "9",
    "municipio_Bacerac": "10",
    "municipio_Bacoachi": "11",
    "municipio_Bácum": "12",
    "municipio_Banámichi": "13",
    "municipio_Baviácora": "14",
    "municipio_Bavispe": "15",
    "municipio_Benjamín Hill": "16",
    "municipio_Caborca": "17",
    "municipio_Cajeme": "18",
    "municipio_Cananea": "19",
    "municipio_Carbó": "20",
    "municipio_Cucurpe": "22",
    "municipio_Cumpas": "23",
    "municipio_Divisaderos": "24",
    "municipio_Empalme": "25",
    "municipio_Etchojoa": "26",
    "municipio_Fronteras": "27",
    "municipio_General Plutarco Elías Calles": "70",
    "municipio_Granados": "28",
    "municipio_Guaymas": "29",
    "municipio_Hermosillo": "30",
    "municipio_Huachinera": "31",
    "municipio_Huásabas": "32",
    "municipio_Huatabampo": "33",
    "municipio_Huépac": "34",
    "municipio_Imuris": "35",
    "municipio_La Colorada": "21",
    "municipio_Naco": "39",
    "municipio_Nácori Chico": "40",
    "municipio_Nacozari de García": "41",
    "municipio_Navojoa": "42",
    "municipio_Nogales": "115",
    "municipio_Onavas": "44",
    "municipio_Opodepe": "45",
    "municipio_Oquitoa": "46",
    "municipio_Pitiquito": "47",
    "municipio_Puerto Peñasco": "48",
    "municipio_Quiriego": "49",
    "municipio_Sahuaripa": "52",
    "municipio_San Felipe de Jesús": "53",
    "municipio_San Ignacio Río Muerto": "72",
    "municipio_San Javier": "54",
    "municipio_San Luis Río Colorado": "55",
    "municipio_San Miguel de Horcasitas": "56",
    "municipio_San Pedro de La Cueva": "57",
    "municipio_Santa Cruz": "59",
    "municipio_Sáric": "60",
    "municipio_Soyopa": "61",
    "municipio_Suaqui Grande": "62",
    "municipio_Tepache": "63",
    "municipio_Trincheras": "64",
    "municipio_Tubutama": "65",
    "municipio_Ures": "66",
    "municipio_Villa Pesqueira": "68",
    "municipio_Yécora": "69",
    "distrito_Cárdenas": "151",
    "distrito_Emiliano Zapata": "153",
    "distrito_Plan Balancan-Tenosique": "154",
    "distrito_Plan Chontalpa": "152",
    "distrito_Villahermosa": "150",
    "municipio_Balancán": "1",
    "municipio_Centla": "3",
    "municipio_Centro": "4",
    "municipio_Comalcalco": "5",
    "municipio_Cunduacán": "6",
    "municipio_Huimanguillo": "8",
    "municipio_Jalapa": "9",
    "municipio_Jalpa de Méndez": "10",
    "municipio_Jonuta": "11",
    "municipio_Macuspana": "12",
    "municipio_Nacajuca": "13",
    "municipio_Paraíso": "14",
    "municipio_Tacotalpa": "15",
    "municipio_Teapa": "16",
    "municipio_Tenosique": "17",
    "distrito_Abasolo": "159",
    "distrito_Control": "157",
    "distrito_Díaz Ordaz": "156",
    "distrito_González": "163",
    "distrito_Jaumave": "161",
    "distrito_Laredo": "155",
    "distrito_Mante": "162",
    "distrito_San Fernando": "158",
    "distrito_Victoria": "160",
    "municipio_Altamira": "3",
    "municipio_Antiguo Morelos": "4",
    "municipio_Burgos": "5",
    "municipio_Casas": "8",
    "municipio_Ciudad Madero": "9",
    "municipio_Cruillas": "10",
    "municipio_El Mante": "21",
    "municipio_González": "12",
    "municipio_Güémez": "13",
    "municipio_Gustavo Díaz Ordaz": "15",
    "municipio_Jaumave": "17",
    "municipio_Llera": "19",
    "municipio_Mainero": "20",
    "municipio_Méndez": "23",
    "municipio_Mier": "24",
    "municipio_Miguel Alemán": "25",
    "municipio_Miquihuana": "26",
    "municipio_Nuevo Laredo": "27",
    "municipio_Nuevo Morelos": "28",
    "municipio_Padilla": "30",
    "municipio_Palmillas": "31",
    "municipio_Reynosa": "32",
    "municipio_Río Bravo": "33",
    "municipio_San Carlos": "34",
    "municipio_Soto La Marina": "37",
    "municipio_Tampico": "38",
    "municipio_Tula": "39",
    "municipio_Valle Hermoso": "40",
    "municipio_Xicoténcatl": "43",
    "distrito_Calpulalpan": "165",
    "distrito_Huamantla": "166",
    "distrito_Tlaxcala": "164",
    "municipio_Acuamanala de Miguel Hidalgo": "22",
    "municipio_Amaxac de Guerrero": "1",
    "municipio_Apetatitlán de Antonio Carvajal": "2",
    "municipio_Apizaco": "5",
    "municipio_Atlangatepec": "3",
    "municipio_Atltzayanca": "4",
    "municipio_Calpulalpan": "6",
    "municipio_Chiautempan": "10",
    "municipio_Contla de Juan Cuamatzi": "18",
    "municipio_Cuapiaxtla": "8",
    "municipio_Cuaxomulco": "9",
    "municipio_El Carmen Tequexquitla": "7",
    "municipio_Españita": "12",
    "municipio_Huamantla": "13",
    "municipio_Hueyotlipan": "14",
    "municipio_Ixtacuixtla de Mariano Matamoros": "15",
    "municipio_Ixtenco": "16",
    "municipio_La Magdalena Tlaltelulco": "48",
    "municipio_Mazatecochco de José María Morelos": "17",
    "municipio_Muñoz de Domingo Arenas": "11",
    "municipio_Nanacamilpa de Mariano Arista": "21",
    "municipio_Natívitas": "23",
    "municipio_Panotla": "24",
    "municipio_Papalotla de Xicohténcatl": "41",
    "municipio_San Damián Texóloc": "49",
    "municipio_San Francisco Tetlanohcan": "50",
    "municipio_San Jerónimo Zacualpan": "51",
    "municipio_San José Teacalco": "52",
    "municipio_San Juan Huactzinco": "53",
    "municipio_San Lorenzo Axocomanitla": "54",
    "municipio_San Lucas Tecopilco": "55",
    "municipio_San Pablo del Monte": "25",
    "municipio_Sanctórum de Lázaro Cárdenas": "20",
    "municipio_Santa Ana Nopalucan": "56",
    "municipio_Santa Apolonia Teacalco": "57",
    "municipio_Santa Catarina Ayometla": "58",
    "municipio_Santa Cruz Quilehtla": "59",
    "municipio_Santa Cruz Tlaxcala": "26",
    "municipio_Santa Isabel Xiloxoxtla": "60",
    "municipio_Teolocholco": "28",
    "municipio_Tepetitla de Lardizábal": "19",
    "municipio_Tepeyanco": "29",
    "municipio_Terrenate": "30",
    "municipio_Tetla de La Solidaridad": "31",
    "municipio_Tetlatlahuca": "32",
    "municipio_Tlaxcala": "33",
    "municipio_Tocatlán": "35",
    "municipio_Totolac": "36",
    "municipio_Tzompantepec": "38",
    "municipio_Xaloztoc": "39",
    "municipio_Xaltocan": "40",
    "municipio_Xicohtzinco": "42",
    "municipio_Yauhquemehcan": "43",
    "municipio_Zacatelco": "44",
    "municipio_Ziltlaltépec de Trinidad Sánchez Santos": "37",
    "distrito_Choapas": "177",
    "distrito_Ciudad Alemán": "174",
    "distrito_Coatepec": "170",
    "distrito_Fortín": "171",
    "distrito_Huayacocotla": "167",
    "distrito_Jáltipan": "176",
    "distrito_La Antigua": "172",
    "distrito_Martínez de la Torre": "169",
    "distrito_Pánuco": "178",
    "distrito_San Andrés Tuxtla": "175",
    "distrito_Tuxpan": "168",
    "distrito_Veracruz": "173",
    "municipio_Acayucan": "3",
    "municipio_Acula": "5",
    "municipio_Acultzingo": "6",
    "municipio_Agua Dulce": "204",
    "municipio_Álamo Temapache": "160",
    "municipio_Alpatláhuac": "8",
    "municipio_Alto Lucero de Gutiérrez Barrios": "9",
    "municipio_Altotonga": "10",
    "municipio_Alvarado": "11",
    "municipio_Amatitlán": "12",
    "municipio_Amatlán de Los Reyes": "14",
    "municipio_Angel R. Cabada": "15",
    "municipio_Apazapan": "17",
    "municipio_Astacinga": "19",
    "municipio_Atlahuilco": "20",
    "municipio_Atzacan": "22",
    "municipio_Atzalan": "23",
    "municipio_Ayahualulco": "25",
    "municipio_Banderilla": "26",
    "municipio_Boca del Río": "28",
    "municipio_Calcahualco": "29",
    "municipio_Camarón de Tejeda": "7",
    "municipio_Camerino Z. Mendoza": "30",
    "municipio_Carlos A. Carrillo": "208",
    "municipio_Carrillo Puerto": "31",
    "municipio_Castillo de Teayo": "157",
    "municipio_Catemaco": "32",
    "municipio_Cazones de Herrera": "33",
    "municipio_Cerro Azul": "34",
    "municipio_Chacaltianguis": "54",
    "municipio_Chalma": "55",
    "municipio_Chiconamel": "56",
    "municipio_Chiconquiaco": "57",
    "municipio_Chicontepec": "58",
    "municipio_Chinameca": "59",
    "municipio_Chinampa de Gorostiza": "60",
    "municipio_Chocamán": "62",
    "municipio_Chontla": "63",
    "municipio_Chumatlán": "64",
    "municipio_Citlaltépetl": "35",
    "municipio_Coacoatzintla": "36",
    "municipio_Coahuitlán": "37",
    "municipio_Coatzacoalcos": "39",
    "municipio_Coatzintla": "40",
    "municipio_Coetzala": "41",
    "municipio_Colipa": "42",
    "municipio_Comapa": "43",
    "municipio_Córdoba": "44",
    "municipio_Cosamaloapan de Carpio": "45",
    "municipio_Cosautlán de Carvajal": "46",
    "municipio_Coscomatepec": "47",
    "municipio_Cosoleacaque": "48",
    "municipio_Cotaxtla": "49",
    "municipio_Coxquihui": "50",
    "municipio_Coyutla": "51",
    "municipio_Cuichapa": "52",
    "municipio_Cuitláhuac": "53",
    "municipio_El Higo": "205",
    "municipio_Espinal": "66",
    "municipio_Filomeno Mata": "67",
    "municipio_Fortín": "68",
    "municipio_Gutiérrez Zamora": "69",
    "municipio_Hidalgotitlán": "70",
    "municipio_Huatusco": "71",
    "municipio_Huayacocotla": "72",
    "municipio_Hueyapan de Ocampo": "73",
    "municipio_Huiloapan de Cuauhtémoc": "74",
    "municipio_Ignacio de La Llave": "75",
    "municipio_Ilamatlán": "76",
    "municipio_Isla": "77",
    "municipio_Ixcatepec": "78",
    "municipio_Ixhuacán de Los Reyes": "79",
    "municipio_Ixhuatlán de Madero": "83",
    "municipio_Ixhuatlán del Café": "80",
    "municipio_Ixhuatlán del Sureste": "82",
    "municipio_Ixhuatlancillo": "81",
    "municipio_Ixmatlahuacan": "84",
    "municipio_Ixtaczoquitlán": "85",
    "municipio_Jalacingo": "86",
    "municipio_Jalcomulco": "88",
    "municipio_Jáltipan": "89",
    "municipio_Jamapa": "90",
    "municipio_Jesús Carranza": "91",
    "municipio_José Azueta": "169",
    "municipio_Juan Rodríguez Clara": "94",
    "municipio_Juchique de Ferrer": "95",
    "municipio_La Antigua": "16",
    "municipio_La Perla": "127",
    "municipio_Landero Y Coss": "96",
    "municipio_Las Choapas": "61",
    "municipio_Las Minas": "107",
    "municipio_Las Vigas de Ramírez": "132",
    "municipio_Lerdo de Tejada": "97",
    "municipio_Maltrata": "99",
    "municipio_Manlio Fabio Altamirano": "100",
    "municipio_Mariano Escobedo": "101",
    "municipio_Martínez de La Torre": "102",
    "municipio_Mecatlán": "103",
    "municipio_Mecayapan": "104",
    "municipio_Medellín": "105",
    "municipio_Miahuatlán": "106",
    "municipio_Misantla": "109",
    "municipio_Mixtla de Altamirano": "110",
    "municipio_Moloacán": "111",
    "municipio_Nanchital de Lázaro Cárdenas del Río": "206",
    "municipio_Naolinco": "112",
    "municipio_Naranjal": "113",
    "municipio_Naranjos Amatlán": "13",
    "municipio_Nautla": "114",
    "municipio_Oluta": "116",
    "municipio_Omealca": "117",
    "municipio_Orizaba": "118",
    "municipio_Otatitlán": "119",
    "municipio_Oteapan": "120",
    "municipio_Ozuluama de Mascareñas": "121",
    "municipio_Pajapan": "122",
    "municipio_Pánuco": "37",
    "municipio_Papantla": "124",
    "municipio_Paso de Ovejas": "126",
    "municipio_Paso del Macho": "125",
    "municipio_Perote": "128",
    "municipio_Platón Sánchez": "129",
    "municipio_Playa Vicente": "130",
    "municipio_Poza Rica de Hidalgo": "131",
    "municipio_Pueblo Viejo": "133",
    "municipio_Puente Nacional": "134",
    "municipio_Rafael Delgado": "135",
    "municipio_Rafael Lucio": "136",
    "municipio_Río Blanco": "138",
    "municipio_Saltabarranca": "139",
    "municipio_San Andrés Tenejapan": "140",
    "municipio_San Andrés Tuxtla": "141",
    "municipio_San Juan Evangelista": "142",
    "municipio_San Rafael": "211",
    "municipio_Santiago Sochiapan": "212",
    "municipio_Santiago Tuxtla": "143",
    "municipio_Sayula de Alemán": "144",
    "municipio_Sochiapa": "146",
    "municipio_Soconusco": "145",
    "municipio_Soledad Atzompa": "147",
    "municipio_Soledad de Doblado": "148",
    "municipio_Soteapan": "149",
    "municipio_Tamalín": "150",
    "municipio_Tamiahua": "151",
    "municipio_Tampico Alto": "152",
    "municipio_Tancoco": "153",
    "municipio_Tantima": "154",
    "municipio_Tantoyuca": "155",
    "municipio_Tatahuicapan de Juárez": "210",
    "municipio_Tatatila": "156",
    "municipio_Tecolutla": "158",
    "municipio_Tehuipango": "159",
    "municipio_Tempoal": "161",
    "municipio_Tenampa": "162",
    "municipio_Tenochtitlán": "163",
    "municipio_Teocelo": "164",
    "municipio_Tepatlaxco": "165",
    "municipio_Tepetlán": "166",
    "municipio_Texcatepec": "170",
    "municipio_Texhuacán": "171",
    "municipio_Texistepec": "172",
    "municipio_Tezonapa": "173",
    "municipio_Tihuatlán": "175",
    "municipio_Tlachichilco": "180",
    "municipio_Tlacojalpan": "176",
    "municipio_Tlacolulan": "177",
    "municipio_Tlacotalpan": "178",
    "municipio_Tlacotepec de Mejía": "179",
    "municipio_Tlalixcoyan": "181",
    "municipio_Tlalnelhuayocan": "182",
    "municipio_Tlaltetela": "24",
    "municipio_Tlapacoyan": "183",
    "municipio_Tlaquilpa": "184",
    "municipio_Tlilapan": "185",
    "municipio_Tonayán": "187",
    "municipio_Totutla": "188",
    "municipio_Tres Valles": "207",
    "municipio_Tuxtilla": "190",
    "municipio_Ursulo Galván": "191",
    "municipio_Uxpanapa": "209",
    "municipio_Vega de Alatorre": "192",
    "municipio_Veracruz": "193",
    "municipio_Villa Aldama": "194",
    "municipio_Xalapa": "87",
    "municipio_Xico": "92",
    "municipio_Yanga": "196",
    "municipio_Yecuatla": "197",
    "municipio_Zentla": "200",
    "municipio_Zongolica": "201",
    "municipio_Zontecomatlán de López Y Fuentes": "202",
    "municipio_Zozocolco de Hidalgo": "203",
    "distrito_Mérida": "179",
    "distrito_Ticul": "180",
    "distrito_Tizimín": "181",
    "distrito_Valladolid": "182",
    "municipio_Abalá": "1",
    "municipio_Acanceh": "2",
    "municipio_Akil": "3",
    "municipio_Baca": "4",
    "municipio_Bokobá": "5",
    "municipio_Buctzotz": "6",
    "municipio_Cacalchén": "7",
    "municipio_Calotmul": "8",
    "municipio_Cansahcab": "9",
    "municipio_Cantamayec": "10",
    "municipio_Celestún": "11",
    "municipio_Cenotillo": "12",
    "municipio_Chacsinkín": "16",
    "municipio_Chankom": "17",
    "municipio_Chapab": "18",
    "municipio_Chemax": "19",
    "municipio_Chichimilá": "21",
    "municipio_Chicxulub Pueblo": "20",
    "municipio_Chikindzonot": "22",
    "municipio_Chocholá": "23",
    "municipio_Chumayel": "24",
    "municipio_Conkal": "13",
    "municipio_Cuncunul": "14",
    "municipio_Cuzamá": "15",
    "municipio_Dzán": "25",
    "municipio_Dzemul": "26",
    "municipio_Dzidzantún": "27",
    "municipio_Dzilam de Bravo": "28",
    "municipio_Dzilam González": "29",
    "municipio_Dzitás": "30",
    "municipio_Dzoncauich": "31",
    "municipio_Espita": "32",
    "municipio_Halachó": "33",
    "municipio_Hocabá": "34",
    "municipio_Hoctún": "35",
    "municipio_Homún": "36",
    "municipio_Huhí": "37",
    "municipio_Hunucmá": "38",
    "municipio_Ixil": "39",
    "municipio_Izamal": "40",
    "municipio_Kanasín": "41",
    "municipio_Kantunil": "42",
    "municipio_Kaua": "43",
    "municipio_Kinchil": "44",
    "municipio_Kopomá": "45",
    "municipio_Mama": "46",
    "municipio_Maní": "47",
    "municipio_Maxcanú": "48",
    "municipio_Mayapán": "49",
    "municipio_Mérida": "50",
    "municipio_Mocochá": "51",
    "municipio_Motul": "52",
    "municipio_Muna": "53",
    "municipio_Muxupip": "54",
    "municipio_Opichén": "55",
    "municipio_Oxkutzcab": "56",
    "municipio_Panabá": "57",
    "municipio_Peto": "58",
    "municipio_Quintana Roo": "60",
    "municipio_Río Lagartos": "61",
    "municipio_Sacalum": "62",
    "municipio_Samahil": "63",
    "municipio_Sanahcat": "64",
    "municipio_Santa Elena": "66",
    "municipio_Seyé": "67",
    "municipio_Sinanché": "68",
    "municipio_Sotuta": "69",
    "municipio_Sucilá": "70",
    "municipio_Sudzal": "71",
    "municipio_Suma": "72",
    "municipio_Tahdziú": "73",
    "municipio_Tahmek": "74",
    "municipio_Teabo": "75",
    "municipio_Tecoh": "76",
    "municipio_Tekal de Venegas": "77",
    "municipio_Tekantó": "78",
    "municipio_Tekax": "79",
    "municipio_Tekit": "80",
    "municipio_Tekom": "81",
    "municipio_Telchac Pueblo": "82",
    "municipio_Telchac Puerto": "83",
    "municipio_Temax": "84",
    "municipio_Temozón": "85",
    "municipio_Tepakán": "86",
    "municipio_Tetiz": "87",
    "municipio_Teya": "88",
    "municipio_Ticul": "89",
    "municipio_Timucuy": "90",
    "municipio_Tinum": "91",
    "municipio_Tixcacalcupul": "92",
    "municipio_Tixkokob": "93",
    "municipio_Tixmehuac": "94",
    "municipio_Tixpéhual": "95",
    "municipio_Tizimín": "96",
    "municipio_Tunkás": "97",
    "municipio_Tzucacab": "98",
    "municipio_Uayma": "99",
    "municipio_Ucú": "100",
    "municipio_Umán": "101",
    "municipio_Valladolid": "102",
    "municipio_Xocchel": "103",
    "municipio_Yaxcabá": "104",
    "municipio_Yaxkukul": "105",
    "municipio_Yobaín": "106",
    "distrito_Concepción del Oro": "188",
    "distrito_Fresnillo": "183",
    "distrito_Jalpa": "190",
    "distrito_Jerez": "187",
    "distrito_Ojo Caliente": "185",
    "distrito_Río Grande": "184",
    "distrito_Tlaltenango": "186",
    "distrito_Zacatecas": "189",
    "municipio_Apozol": "1",
    "municipio_Apulco": "2",
    "municipio_Atolinga": "3",
    "municipio_Calera": "5",
    "municipio_Cañitas de Felipe Pescador": "6",
    "municipio_Chalchihuites": "9",
    "municipio_Concepción del Oro": "7",
    "municipio_El Plateado de Joaquín Amaro": "15",
    "municipio_El Salvador": "41",
    "municipio_Fresnillo": "10",
    "municipio_Genaro Codina": "12",
    "municipio_General Enrique Estrada": "13",
    "municipio_General Francisco R. Murguía": "14",
    "municipio_General Pánfilo Natera": "16",
    "municipio_Huanusco": "18",
    "municipio_Jalpa": "19",
    "municipio_Jerez": "20",
    "municipio_Jiménez del Teul": "21",
    "municipio_Juan Aldama": "22",
    "municipio_Juchipila": "23",
    "municipio_Luis Moya": "25",
    "municipio_Mazapil": "26",
    "municipio_Mezquital del Oro": "28",
    "municipio_Miguel Auza": "29",
    "municipio_Momax": "30",
    "municipio_Monte Escobedo": "31",
    "municipio_Moyahua de Estrada": "33",
    "municipio_Nochistlán de Mejía": "34",
    "municipio_Noria de Ángeles": "35",
    "municipio_Ojocaliente": "36",
    "municipio_Pinos": "38",
    "municipio_Río Grande": "39",
    "municipio_Sain Alto": "40",
    "municipio_Santa María de La Paz": "58",
    "municipio_Sombrerete": "42",
    "municipio_Susticacán": "43",
    "municipio_Tabasco": "44",
    "municipio_Tepechitlán": "45",
    "municipio_Tepetongo": "46",
    "municipio_Teúl de González Ortega": "47",
    "municipio_Tlaltenango de Sánchez Román": "48",
    "municipio_Trancoso": "57",
    "municipio_Trinidad García de La Cadena": "11",
    "municipio_Valparaíso": "49",
    "municipio_Vetagrande": "50",
    "municipio_Villa de Cos": "51",
    "municipio_Villa García": "52",
    "municipio_Villa González Ortega": "53",
    "municipio_Villanueva": "55",
    "municipio_Zacatecas": "56"
}